# Stock Price Forecasting with Machine Learning

## 0. Preliminary workings

### 0.1 Uploading necessary packages

In [1]:
import os
import numpy as np
from scipy.stats import norm, t, uniform
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score
from tqdm import tqdm
from datetime import datetime as dt

from linearmodels.panel import PanelOLS
import statsmodels.api as sm

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import ParameterGrid

from sklearn.linear_model import HuberRegressor
from sklearn.cross_decomposition import PLSRegression
import time
import warnings
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import random
from scipy.optimize import minimize
from functools import partial
from numpy.linalg import svd
from sklearn.linear_model import ElasticNet
from group_lasso import GroupLasso
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

import random
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Input, Dense, BatchNormalization
from keras.regularizers import L1L2
from keras.optimizers import Adam

warnings.filterwarnings("ignore")
random.seed(123)

C:\Users\dguse\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\dguse\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\dguse\anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


### 0.2.1 Defining few functions (may not be relevant anymore)

In [2]:
def fw1(x):
    # Find the maximum location of a vector
    maximum = np.max(x)
    p = np.where(x == maximum)[0]
    if len(p) > 1:
        p = p[0]
    return p

def pls(X, y, A):
    """
    Partial Least Squares (PLS) regression
    
    Parameters:
    X : array-like, shape (n_samples, n_features)
        Training data.
    y : array-like, shape (n_samples,)
        Target values.
    A : int
        Number of components.
        
    Returns:
    B : array, shape (n_features, A)
        Coefficients for each component.
    """
    # Initialize arrays to store intermediate results
    s = X.T.dot(y)
    R = np.zeros((X.shape[1], A))
    TT = np.zeros((X.shape[0], A))
    P = np.zeros((X.shape[1], A))
    U = np.zeros((X.shape[0], A))
    V = np.zeros((X.shape[1], A))
    B = np.zeros((X.shape[1], A))
    Q = np.zeros((1, A))

    # Iterate through each component
    for i in range(A):
        # Calculate loading vector for X and Y
        q = s.T.dot(s)
        r = s.dot(q)
        t = X.dot(r)
        t = t - np.mean(t)
        normt = np.sqrt(t.T.dot(t))
        t = t / normt
        r = r / normt
        p = X.T.dot(t)
        q = y.T.dot(t)
        u = y * q
        v = p

        # Calculate deflation
        if i > 0:
            v = v - V[:, :i+1].dot(V[:, :i+1].T.dot(p))
            u = u - TT[:, :i+1].dot(TT[:, :i+1].T.dot(u))
        v = v / np.sqrt(v.T.dot(v))
        s = s - v.dot(v.T.dot(s))

        # Store results for current component
        R[:, i] = r
        TT[:, i] = t
        P[:, i] = p
        U[:, i] = u
        V[:, i] = v
        Q[:, i] = q

    # Reconstruct the coefficient matrix B
    for i in range(A - 1):
        C = R[:, :i+1].dot(Q[:, :i+1].T)
        B[:, i+1] = C[:, 0]

    return B

def soft_threshodl(groups, nc, w, mu):
    """
    Soft thresholding operator
    
    Parameters:
    groups : int
        Not used in the function, placeholder for the MATLAB code.
    nc : int
        Not used in the function, placeholder for the MATLAB code.
    w : array-like
        Input array.
    mu : float
        Threshold parameter.
    
    Returns:
    val : array-like
        Soft thresholded array.
    """
    val = np.sign(w) * np.maximum(np.abs(w) - mu, 0)
    return val

def lossh(y, yhat, mu):
    """
    Loss function for proximalH
    
    Parameters:
    y : array-like
        True target values.
    yhat : array-like
        Predicted values.
    mu : float
        Threshold parameter.
    
    Returns:
    m : float
        Loss value.
    """
    r = np.abs(yhat - y)
    l = np.zeros(len(r))
    ind = (r > mu)
    l[ind] = 2 * mu * r[ind] - mu * mu
    ind = (r <= mu)
    l[ind] = r[ind] * r[ind]
    m = np.mean(l)
    return m

def f_gradh(w, X, y, mu):
    """
    Gradient of the loss function for proximalH
    
    Parameters:
    w : array-like
        Coefficients.
    X : array-like
        Training data.
    y : array-like
        Target values.
    mu : float
        Threshold parameter.
    
    Returns:
    grad : array-like
        Gradient.
    """
    r = np.dot(X, w) - y
    ind0 = np.where(np.abs(r) <= mu)[0]
    ind1 = np.where(r > mu)[0]
    indf1 = np.where(r < -mu)[0]
    grad = np.dot(X[ind0, :].T, np.dot(X[ind0, :], w) - y[ind0]) + mu * np.dot(X[ind1, :].T, np.ones(len(ind1))) - mu * np.dot(X[indf1, :].T, np.ones(len(indf1)))
    return grad

def proximalH(groups, nc, xtest, mtrain, ytest, w, X, y, mu, tol, L, l2, func):
    """
    Proximal operator using accelerated proximal gradient descent
    
    Parameters:
    groups : int
        Not used in the function, placeholder for the MATLAB code.
    nc : int
        Not used in the function, placeholder for the MATLAB code.
    xtest : array-like
        Test data.
    mtrain : float
        Mean of the training target values.
    ytest : array-like
        Test target values.
    w : array-like
        Initial guess of the coefficients.
    X : array-like
        Training data.
    y : array-like
        Target values.
    mu : float
        Threshold parameter.
    tol : float
        Tolerance parameter for convergence.
    L : float
        Lipschitz constant.
    l2 : float
        Regularization parameter.
    func : function
        Soft thresholding function.
    
    Returns:
    a : array-like
        Final coefficients after proximal gradient descent.
    """
    dim = X.shape[1]
    max_iter = 3000
    gamma = 1 / L
    l1 = l2
    v = w.copy()
    yhatbig1 = np.dot(xtest, w) + mtrain
    r20 = lossh(yhatbig1, ytest, mu)
    
    for t in range(max_iter):
        vold = v.copy()
        w_perv = w.copy()
        w = v - gamma * f_gradh(v, X, y, mu)
        mu1 = l1 * gamma
        w = func(groups, nc, w, mu1)
        v = w + t / (t + 3) * (w - w_perv)
        
        if np.sum((v - vold) ** 2) < (np.sum(vold ** 2) * tol) or np.sum(np.abs(v - vold)) == 0:
            break
    
    return v

def proximal(groups, nc, XX, XY, tol, L, l2, func):
    '''
    groups: Group membership of features
    nc: Number of classes
    XX: Matrix of features
    XY: Matrix of labels
    tol: Tolerance for convergence
    L: Lipschitz constant
    l2: L2 regularization parameter
    func: Proximal operator function
    '''
    dim = XX.shape[0]
    max_iter = 30000
    gamma = 1 / L
    l1 = l2
    w = np.zeros(dim)
    v = w.copy()

    for t in range(max_iter):
        vold = v.copy()
        w_prev = w.copy()
        w = v - gamma * f_grad(XX, XY, v)
        w = func(groups, nc, w, l1 * gamma)
        v = w + t / (t + 3) * (w - w_prev)
        if (np.sum(np.power(v - vold, 2)) < (np.sum(np.power(vold, 2)) * tol)) or (np.sum(np.abs(v - vold)) == 0):
            break

    return v

def f_grad(XX, XY, w):
    """
    Gradient of the objective function.

    Parameters:
    XX (array): Design matrix.
    XY (array): Target values.
    w (array): Coefficients.

    Returns:
    grad (array): Gradient.
    """
    grad = np.dot(XX, w) - XY
    return grad

def soft_threshodr(groups, nc, w, mu):
    """
    Soft thresholding function for ridge regularization.

    Parameters:
    groups (array): Groups.
    nc (int): Number of components.
    w (array): Coefficients.
    mu (float): Threshold parameter.

    Returns:
    val (array): Updated coefficients after soft thresholding.
    """
    val = w / (1 + mu)
    return val

import numpy as np

def cut_knots_degree2(x, n, th):
    '''
    x: Input matrix
    n: Degree of the polynomial
    th: Threshold values for knot points
    '''
    a, b = x.shape  # Get the dimensions of the input matrix
    resultfinal = np.zeros((a, b * (n + 1)))  # Initialize the final result matrix

    for i in range(b):
        xcut = x[:, i]  # Extract a column of the input matrix
        xcutnona = np.copy(xcut)  # Create a copy of the column
        xcutnona[np.isnan(xcutnona)] = 0  # Replace NaN values with 0
        index = ((1 - 1 * np.isnan(xcut)) == 1)  # Find non-NaN indices

        t = th[:, i]  # Get the threshold values for this column

        x1 = xcutnona
        resultfinal[:, (n + 1) * i - n] = x1 - np.mean(x1)  # Store the original values

        x1 = np.power(xcutnona - t[0], 2)
        resultfinal[:, (n + 1) * i - n + 1] = x1 - np.mean(x1)  # Store the squared differences with the first threshold

        for j in range(n - 1):
            x1 = np.power(xcutnona - t[j + 1], 2) * (xcutnona >= t[j + 1])  # Calculate squared differences with subsequent thresholds
            resultfinal[:, (n + 1) * i - n + 1 + j] = x1 - np.mean(x1)  # Store the result in the final matrix

    return resultfinal

def soft_threshode(groups, nc, w, mu):
    '''
    groups: Group membership of features
    nc: Number of classes
    w: Input vector
    mu: Threshold parameter
    '''
    # Apply soft thresholding operation element-wise
    val = np.sign(w) * np.maximum(np.abs(w) - 0.5 * mu, 0) / (1 + 0.5 * mu)
    
    return val

def soft_threshodg(groups, nc, w, mu):
    '''
    groups: Group membership of features
    nc: Number of classes
    w: Input vector
    mu: Threshold parameter
    '''
    w1 = np.copy(w)  # Create a copy of the input vector
    for i in range(1, nc + 1):  # Iterate over the number of classes
        ind = (groups == i)  # Identify indices corresponding to the current class
        wg = w1[ind]  # Extract the values of w corresponding to the current class
        nn = len(wg)  # Get the number of elements in the current class
        n2 = np.sqrt(np.sum(wg ** 2))  # Calculate the L2 norm of the current class
        if n2 <= mu:  # Check if the L2 norm is less than or equal to the threshold parameter mu
            w1[ind] = np.zeros(nn)  # Set the values of w corresponding to the current class to zero
        else:
            w1[ind] = wg - mu * wg / n2  # Apply the soft thresholding operation to the values of w corresponding to the current class
    return w1  # Return the updated vector w

### 0.2.2. New set of functions

In [3]:
# Loss Function
# Huber objective function
def huber(actual, predicted, xi):
    actual, predicted = np.array(actual).flatten(), np.array(predicted).flatten()
    resid = actual - predicted
    huber_loss = np.where(np.abs(resid)<=xi, diff**2, 2*xi*np.abs(resid)-xi**2)
    return np.mean(huber_loss)

# Scoring Function
# out-of-sample R squared
def R_oos(actual, predicted):
    actual, predicted = np.array(actual), np.array(predicted).flatten()
    predicted = np.where(predicted<0,0,predicted)
    return 1 - (np.dot((actual-predicted),(actual-predicted)))/(np.dot(actual,actual))

# Validation Function
def val_fun(model, params: dict, X_trn, y_trn, X_vld, y_vld, illustration=True, sleep=0, is_NN=False):
    best_ros = None
    lst_params = list(ParameterGrid(params))
    for param in lst_params:
        if best_ros == None:
            if is_NN:
                mod = model().set_params(**param).fit(X_trn, y_trn, X_vld, y_vld)
            else:
                mod = model().set_params(**param).fit(X_trn, y_trn)
            best_mod = mod
            y_pred = mod.predict(X_vld)
            best_ros = R_oos(y_vld, y_pred)
            best_param = param
            if illustration:
                print(f'Model with params: {param} finished.')
                print(f'with out-of-sample R squared on validation set: {best_ros*100:.5f}%')
                print('*'*60)
        else:
            time.sleep(sleep)
            if is_NN:
                mod = model().set_params(**param).fit(X_trn, y_trn, X_vld, y_vld)
            else:
                mod = model().set_params(**param).fit(X_trn, y_trn)
            y_pred = mod.predict(X_vld)
            ros = R_oos(y_vld, y_pred)
            if illustration:
                print(f'Model with params: {param} finished.')
                print(f'with out-of-sample R squared on validation set: {ros*100:.5f}%')
                print('*'*60)
            if ros > best_ros:
                best_ros = ros
                best_mod = mod
                best_param = param
    if illustration:
        print('\n'+'#'*60)
        print('Tuning process finished!!!')
        print(f'The best setting is: {best_param}')
        print(f'with R2oos {best_ros*100:.2f}% on validation set.')
        print('#'*60)
    return best_mod
    

# Pairwise Comparison
# Diebold-Mariano test statistics

# Evaluation Output
def evaluate(actual, predicted, insample=False):
    if insample:
        print('*'*15+'In-Sample Metrics'+'*'*15)
        print(f'The in-sample R2 is {r2_score(actual,predicted)*100:.2f}%')
        print(f'The in-sample MSE is {mean_squared_error(actual,predicted):.3f}')
    else:
        print('*'*15+'Out-of-Sample Metrics'+'*'*15)
        print(f'The out-of-sample R2 is {R_oos(actual,predicted)*100:.2f}%')
        print(f'The out-of-sample MSE is {mean_squared_error(actual,predicted):.3f}')

In [4]:
class PCRegressor:
    
    def __init__(self,n_PCs=1,loss='mse'):
        self.n_PCs = n_PCs
        if loss not in ['huber','mse']:
            raise AttributeError(
            f"The loss should be either 'huber' or 'mse', but {loss} is given"
            )
        else:
            self.loss = loss
        
    def set_params(self, **params):
        for param in params.keys():
            setattr(self, param, params[param])
        return self
        
    def fit(self,X,y):
        X = np.array(X)
        N,K = X.shape
        y = np.array(y_trn).reshape((N,1))
        self.mu = np.mean(X,axis=0).reshape((1,K))
        self.sigma = np.std(X,axis=0).reshape((1,K))
        self.sigma = np.where(self.sigma==0,1,self.sigma)
        X = (X-self.mu)/self.sigma
        pca = PCA()
        X = pca.fit_transform(X)[:,:self.n_PCs]
        self.pc_coef = pca.components_.T[:,:self.n_PCs]
        if self.loss == 'mse':
            self.model = LinearRegression().fit(X,y)
        else:
            self.model = HuberRegressor().fit(X,y)
        return self
    
    def predict(self,X):
        X = np.array(X)
        X = (X-self.mu)/self.sigma
        X = X @ self.pc_coef
        return self.model.predict(X)

In [5]:
# mse
def mse(actual, predicted):
    actual, predicted = np.array(actual).flatten(), np.array(predicted).flatten()
    resid = actual - predicted
    return np.mean(resid**2)

# huber objective function
def huber(actual, predicted, xi):
    actual, predicted = np.array(actual).flatten(), np.array(predicted).flatten()
    resid = actual - predicted
    huber_loss = np.where(np.abs(resid)<=xi, diff**2, 2*xi*np.abs(resid)-xi**2)
    return np.mean(huber_loss)

# proximal operator
def prox(theta,lmd,rho,gamma):
    return (1/(1+lmd*gamma*rho))*softhred(theta,(1-rho)*gamma*lmd)

# soft-thresholding operator
def softhred(x,mu):
    x = np.where(np.abs(x)<=mu, 0, x)
    x = np.where((np.abs(x)>mu) & (x>0), x-mu, x)
    x = np.where((np.abs(x)>mu) & (x<0), x+mu, x)
    return x

# penalized mse
def mse_pnl(theta, X, y, lmd, rho):
    K = X.shape[1]
    X = np.array(X)
    N = len(y)
    y = np.array(y).reshape((N,1))
    resid = y - X@theta.reshape((K,1))
    return np.mean(resid**2) + lmd*(1-rho)*np.sum(np.abs(theta)) + 0.5*lmd*rho*np.sum(theta**2)

# penalized huber objective function
def huber_pnl(theta, X, y, xi, lmd, rho):
    K = X.shape[1]
    X = np.array(X)
    N = len(y)
    y = np.array(y).reshape((N,1))
    resid = y - X@theta.reshape((K,1))
    huber_loss = np.where(np.abs(resid)<=xi, resid**2, 2*xi*np.abs(resid)-xi**2)
    return np.mean(huber_loss) + lmd*(1-rho)*np.sum(np.abs(theta)) + 0.5*lmd*rho*np.sum(theta**2)

# gradient of mse
def grad_mse(theta, X, y, lmd, rho):
    K = X.shape[1]
    X = np.array(X)
    N = len(y)
    y = np.array(y).reshape((N,1))
    theta = np.array(theta).reshape((K,1))
    grad = (X.T @ (y - X@theta))/N + lmd*(1-rho)*np.where(theta>0,1,-1) + lmd*rho*theta
    return grad.flatten()

# gradient of huber loss
def grad_huber(theta, X, y, xi, lmd, rho):
    K = X.shape[1]
    X = np.array(X)
    N = len(y)
    y = np.array(y).reshape((N,1))
    theta = np.array(theta).reshape((K,1))
    resid = y - X@theta
    ind_m = np.where(np.abs(resid)<=xi)
    ind_u = np.where(resid>xi)
    ind_l = np.where(resid< -xi)
    try:
        grad_m = X[ind_m].T @ (y[ind_m] - X[ind_m]@theta)
    except:
        grad_m = np.zeros((K,1))
    try:
        grad_u = 2*xi* X[ind_u].T@np.ones((len(ind_u[0]),1))
    except:
        grad_u = np.zeros((K,1))
    try:
        grad_l = -2*xi* X[ind_l].T@np.ones((len(ind_l[0]),1))
    except:
        grad_l = np.zeros((K,1))
    grad = (grad_m+grad_u+grad_l)/N + lmd*(1-rho)*np.where(theta>0,1,-1) + lmd*rho*theta
    return grad.flatten()

# Elastic Net
class ENet:
    
    def __init__(
        self, lmd, rho, xi=1.35, loss='huber', random_state=None, fit_intercept=True
    ):
        self.lmd = lmd
        self.rho = rho
        self.xi = xi
        self.random_state = random_state
        self.fit_intercept = fit_intercept
        if loss not in ['huber','mse']:
            raise AttributeError(
            f"The loss should be either 'huber' or 'mse', but {loss} is given"
            )
        else:
            self.loss = loss
            
    def set_params(self, **params):
        for param in params.keys():
            setattr(self, param, params[param])
        return self
            
    def fit(self, X, y):
        K = X.shape[1]
        X = np.array(X)
        N = len(y)
        y = np.array(y).reshape((N,1))
        if self.fit_intercept:
            K += 1
            X = np.concatenate((np.ones((N,1)),X),axis=1)
        # initialize theta
        if self.random_state != None:
            np.random.seed(self.random_state)
            theta = np.random.uniform(K)
        else:
            theta = np.zeros(K)
        
        if self.loss == 'huber':
            res = minimize(
                partial(huber_pnl, X=X, y=y, xi=self.xi, lmd=self.lmd, rho=self.rho), theta,
                method='nelder-mead',
                #method='BFGS',
                #jac = partial(grad_huber, X=X, y=y, xi=self.xi, lmd=self.lmd, rho=self.rho),
                options = {'disp': True}
            )
        else:
            res = minimize(
                partial(mse_pnl, X=X, y=y, lmd=self.lmd, rho=self.rho), theta, 
                method='nelder-mead',
                #method='BFGS',
                #jac = partial(grad_mse, X=X, y=y, lmd=self.lmd, rho=self.rho),
                options = {'disp': True}
            )
        
        self.theta = res.x.reshape((K,1))
        return self
    
    def predict(self, X):
        X = np.array(X)
        N = X.shape[0]
        if self.fit_intercept:
            X = np.concatenate((np.ones((N,1)),X),axis=1)
        return X@self.theta

In [6]:
def flatten(l):
    return [item for sublist in l for item in sublist]

def SplineTransform(data,knots=3):
    spline_data = pd.DataFrame(np.ones((data.shape[0],1)),index=data.index,columns=['const'])
    for i in data.columns:
        i_dat = data.loc[:,i]
        i_sqr = i_dat**2
        i_cut, bins = pd.cut(i_dat, 3, right=True, ordered=True, retbins=True)
        i_dum = pd.get_dummies(i_cut)
        for j in np.arange(knots):
            i_dum.iloc[:,j] = i_dum.iloc[:,j]*((i_dat-bins[j])**2)
        i_dum.columns = [f"{i}_{k}" for k in np.arange(1,knots+1)]
        spline_data = pd.concat((spline_data,i_dat,i_dum),axis=1)
    return spline_data

class GLMRegression:
    
    def __init__(self,knots=3,lmd=1e-4,l1_reg=1e-4,random_state=12308):
        self.knots = knots
        self.lmd = lmd
        self.random_state = random_state
        self.l1_reg = l1_reg
        
    def set_params(self, **params):
        for param in params.keys():
            setattr(self, param, params[param])
        return self
    
    def fit(self,X,y):
        groups = [0]+flatten([list(np.repeat(i,self.knots+1))[:] for i in np.arange(1,X.shape[1]+1)])
        X = SplineTransform(X)
        self.mod = GroupLasso(
            groups=groups,group_reg=self.lmd,l1_reg=self.l1_reg,
            fit_intercept=False,random_state=self.random_state
        )
        self.mod = self.mod.fit(X,y)
        return self
    
    def predict(self,X):
        X = SplineTransform(X)
        return self.mod.predict(X)

### 0.3 Merging datasets

In [ ]:
c1 = pd.read_csv('datashare.csv')
r1 = pd.read_csv('funny_data.csv')
some = pd.read_csv('small.csv')

In [ ]:
c1['DATE'] = pd.to_datetime(c1['DATE'],format='%Y%m%d')+pd.offsets.MonthEnd(0)
r1['DATE'] = r1['date']
r1['DATE'] = pd.to_datetime(r1['DATE'])+pd.offsets.MonthEnd(0)

In [ ]:
c1.to_csv('./Simu/Simu_p50/c1_1.csv', index=False)
r1.to_csv('./Simu/Simu_p50/r1_1.csv', index=False)

In [7]:
c1 = pd.read_csv("./Simu/Simu_p50/c1_1.csv")
r1 = pd.read_csv("./Simu/Simu_p50/r1_1.csv")

In [8]:
stock = pd.Categorical(c1['permno'])
c1 = c1.set_index(["permno", "DATE"])
c1['stock'] = stock

In [9]:
stock = pd.Categorical(r1["PERMNO"])
r1 = r1.set_index(["PERMNO", "date"])
r1['stock'] = stock

In [10]:
c1.index.names = ['permno', 'date']
r1.index.names = ['permno', 'date']

In [11]:
c1 = c1.sort_index(level=['permno', 'date'])
r1 = r1.sort_index(level=['permno', 'date'])

In [12]:
new_date_index = pd.to_datetime(r1.index.get_level_values(level = "date"))
new_index = pd.MultiIndex.from_tuples(zip(r1['stock'], new_date_index), names=['permno', 'date'])
r1.index = new_index

new_date_index = pd.to_datetime(c1.index.get_level_values(level = "date"))
new_index = pd.MultiIndex.from_tuples(zip(c1['stock'], new_date_index), names=['permno', 'date'])
c1.index = new_index

---------------------

#### Set the prediod of prediction

In [13]:
starting = pd.to_datetime('1980-12-31')
ending = pd.to_datetime('1990-01-01')

In [14]:
c1_small = c1[(c1.index.get_level_values(level = "date") >= starting) & (c1.index.get_level_values(level = "date") <= ending)]
r1_small = r1[(r1.index.get_level_values(level = "date") >= starting) & (r1.index.get_level_values(level = "date") <= ending)]

---------------

In [15]:
b1 = c1_small.merge(r1_small, left_index=True, right_on=['permno', 'date'])

### 0.3.1 Cleaning the data from bad tickers

In [16]:
b1['TICKER'] = pd.Categorical(b1['TICKER'])
additional = pd.DataFrame(b1['TICKER'].value_counts())
bad_spisok = additional[additional['count'] <= 5].index

b1 = b1[~b1['TICKER'].isin(bad_spisok)]

In [17]:
bad_spisok

CategoricalIndex(['MVICV', 'CMPX', 'MRCH', 'BTEL', 'TSIN', 'NEG', 'TCS',
                  'KECO', 'TSIX', 'TCO',
                  ...
                  'TAREF', 'LCCQ', 'DGII', 'SYT', 'PRPL', 'SYSE', 'BDN',
                  'DIND', 'NNH', 'MOU'],
                 categories=['A', 'AA', 'AAA', 'AAC', ..., 'ZUSC', 'ZY', 'ZYAD', 'ZYMS'], ordered=False, dtype='category', name='TICKER', length=774)

### 0.4 Filling missing values

In [18]:
b1 = b1.drop(columns = ['stock_x', 'stock_y', 'DATE', 'TICKER', 'RETX'])
b1['RET'] = pd.to_numeric(b1['RET'], errors = "coerce")

characteristics = list(set(b1.columns).difference({'SHROUT','mve0','sic2','prc', "SICCD"}))
b1.isnull().sum()

mvel1        380
beta       57369
betasq     57369
chmom      48063
dolvol     54582
           ...  
ASKHI          0
PRC         2359
VOL        40752
RET         2360
SPREAD    279887
Length: 104, dtype: int64

In [19]:
for index, value in enumerate(b1['RET']):
    if isinstance(value, str):
        b1.at[index, 'RET'] = np.nan

In [20]:
%%time
# fill na with cross-sectional median
for ch in characteristics:
     b1[ch] = b1.groupby('date')[ch].transform(lambda x: x.fillna(x.median()))

CPU times: total: 10.2 s
Wall time: 10.3 s


In [21]:
for ch in characteristics:
     b1[ch] = b1[ch].fillna(0)
    
b1.columns[b1.isnull().sum()!=0]

Index(['sic2'], dtype='object')

### 0.5 Adjusting Industrical Classification variable 

In [22]:
def get_sic_dummies(data_ch):
    sic_dummies = pd.get_dummies(b1['sic2'].fillna(999).astype(int),prefix='sic').drop('sic_999',axis=1)
    b1_d = pd.concat([b1,sic_dummies],axis=1)
    b1_d.drop(['sic2'],inplace=True,axis=1)
    b1_d.drop(['SICCD'],inplace=True,axis=1)
    return b1_d

In [23]:
b1_d = get_sic_dummies(b1)
b1_d.dtypes.value_counts()

float64    101
bool        71
int64        1
Name: count, dtype: int64

### 0.6 Train-Test Split

In [24]:
from sklearn.preprocessing import MinMaxScaler

index_co = b1_d.index
features = list(set(b1_d.columns).difference({'permno','DATE','RET'}))
X = MinMaxScaler((-1,1)).fit_transform(b1_d[features])
X = pd.DataFrame(X, columns=features, index=index_co)
y = b1_d['RET']

In [25]:

stdt_vld = starting + (ending - starting)/3
stdt_tst = starting + 2*(ending - starting)/3

X_trn = X[X.index.get_level_values(level = "date") <= stdt_vld]
y_trn = y[y.index.get_level_values(level = "date") <= stdt_vld]

X_vld = X[(X.index.get_level_values(level = "date") >= stdt_vld) & (X.index.get_level_values(level = "date") <= stdt_tst)]
y_vld = y[(y.index.get_level_values(level = "date") >= stdt_vld) & (y.index.get_level_values(level = "date") <= stdt_tst)]

X_tst = X[X.index.get_level_values(level = "date") >= stdt_tst]
y_tst = y[y.index.get_level_values(level = "date") >= stdt_tst]

-----------------------

## 1.0 Linear Model (Old)

In [26]:
# PooledOLS - this is what I need to do. Plus add the Huber Func
exog_vars = list(set(b1.columns).difference({'sic2','RET', "SICCD"}))
exog = sm.add_constant(b1[exog_vars])
mod = PanelOLS(b1.RET, exog, check_rank=False)
pooled_res = mod.fit()
print(pooled_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:                    RET   R-squared:                        0.0133
Estimator:                   PanelOLS   R-squared (Between):             -0.0151
No. Observations:              471892   R-squared (Within):               0.0145
Date:                Wed, Apr 17 2024   R-squared (Overall):              0.0133
Time:                        20:53:16   Log-likelihood                 1.813e+05
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      62.758
Entities:                       11089   P-value                           0.0000
Avg Obs:                       42.555   Distribution:              F(101,471790)
Min Obs:                       1.0000                                           
Max Obs:                       76.000   F-statistic (robust):             62.758
                            

In [27]:
# Plot some stocks and check how they behave. Check the data consostency and clear bad stocks (write them down in additional list.
# Create a table with each bad ticker and associated problem (and check the paper for the advice)

## 1.1 Linear Models (New)

### 1.1.1 OLS

In [28]:
%%time
### Simple OLS with MSE as a loss function

# OLS with all features
OLS = LinearRegression().fit(X_trn,y_trn)
evaluate(y_trn, OLS.predict(X_trn), insample=True)
evaluate(y_tst, OLS.predict(X_tst))

***************In-Sample Metrics***************
The in-sample R2 is 4.80%
The in-sample MSE is 0.027
***************Out-of-Sample Metrics***************
The out-of-sample R2 is -0.05%
The out-of-sample MSE is 3693715471123923456.000
CPU times: total: 12.3 s
Wall time: 2.56 s


In [29]:
%%time
### OLS with a Huber loss function

# OLS by Huber robust objective function with all features
epsilon = np.max(((y_trn-OLS.predict(X_trn)).quantile(.999),1))
OLS_H = HuberRegressor(epsilon=epsilon).fit(X_trn,y_trn)
evaluate(y_trn, OLS_H.predict(X_trn), insample=True)
evaluate(y_tst, OLS_H.predict(X_tst))

***************In-Sample Metrics***************
The in-sample R2 is 2.08%
The in-sample MSE is 0.028
***************Out-of-Sample Metrics***************
The out-of-sample R2 is -0.18%
The out-of-sample MSE is 0.027
CPU times: total: 38.1 s
Wall time: 19.4 s


In [30]:
# OLS with preselected size, bm, and momentum covariates
features_3 = ['mvel1','bm','mom1m','mom6m','mom12m','mom36m']
OLS_3 = LinearRegression().fit(X_trn[features_3],y_trn)

### 1.1.2 PLS

In [31]:
%%time

params = {'n_components': [1, 5, 10, 50]}
PLS = val_fun(PLSRegression,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld)

Model with params: {'n_components': 1} finished.
with out-of-sample R squared on validation set: -1.59834%
************************************************************
Model with params: {'n_components': 5} finished.
with out-of-sample R squared on validation set: -5.27974%
************************************************************
Model with params: {'n_components': 10} finished.
with out-of-sample R squared on validation set: -5.46342%
************************************************************
Model with params: {'n_components': 50} finished.
with out-of-sample R squared on validation set: -5.43570%
************************************************************

############################################################
Tuning process finished!!!
The best setting is: {'n_components': 1}
with R2oos -1.60% on validation set.
############################################################
CPU times: total: 26.2 s
Wall time: 12.3 s


In [32]:
%%time
pls_pred_is = PLS.predict(X_trn)
pls_pred_os = PLS.predict(X_tst)
evaluate(y_trn, pls_pred_is, insample=True) 
evaluate(y_tst, pls_pred_os)

***************In-Sample Metrics***************
The in-sample R2 is 2.17%
The in-sample MSE is 0.028
***************Out-of-Sample Metrics***************
The out-of-sample R2 is -2.34%
The out-of-sample MSE is 0.027
CPU times: total: 422 ms
Wall time: 269 ms


### 1.1.3 PCR

In [33]:
%%time

params = {'n_PCs':[1,3,5,7,10,50],'loss':['mse','huber']}
PCR = val_fun(PCRegressor,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,sleep=3)

Model with params: {'loss': 'mse', 'n_PCs': 1} finished.
with out-of-sample R squared on validation set: -0.62917%
************************************************************
Model with params: {'loss': 'mse', 'n_PCs': 3} finished.
with out-of-sample R squared on validation set: -0.61700%
************************************************************
Model with params: {'loss': 'mse', 'n_PCs': 5} finished.
with out-of-sample R squared on validation set: -0.66151%
************************************************************
Model with params: {'loss': 'mse', 'n_PCs': 7} finished.
with out-of-sample R squared on validation set: -0.66961%
************************************************************
Model with params: {'loss': 'mse', 'n_PCs': 10} finished.
with out-of-sample R squared on validation set: -0.80923%
************************************************************
Model with params: {'loss': 'mse', 'n_PCs': 50} finished.
with out-of-sample R squared on validation set: -1.97006%
***

In [34]:
%%time
evaluate(y_trn, PCR.predict(X_trn), insample=True) 
evaluate(y_tst, PCR.predict(X_tst))

***************In-Sample Metrics***************
The in-sample R2 is -0.94%
The in-sample MSE is 0.029
***************Out-of-Sample Metrics***************
The out-of-sample R2 is 0.21%
The out-of-sample MSE is 0.027
CPU times: total: 984 ms
Wall time: 485 ms


### 1.1.4 Elastic Net

In [35]:
%%time
random.seed(12308)
EN_my_mse_rdm = ENet(lmd=.01,rho=.5,loss='huber').fit(X_trn,y_trn)

Optimization terminated successfully.
         Current function value: 0.027460
         Iterations: 21202
         Function evaluations: 22509
CPU times: total: 43min 35s
Wall time: 21min 41s


In [36]:
%%time
evaluate(y_trn, EN_my_mse_rdm.predict(X_trn), insample=True) 
evaluate(y_tst, EN_my_mse_rdm.predict(X_tst))

***************In-Sample Metrics***************
The in-sample R2 is 0.71%
The in-sample MSE is 0.028
***************Out-of-Sample Metrics***************
The out-of-sample R2 is -0.26%
The out-of-sample MSE is 0.026
CPU times: total: 281 ms
Wall time: 283 ms


### 1.1.5 GLM

In [37]:
%%time

params = {
    'knots':[3],
    'lmd':[1e-4,1e-1],#list(np.linspace(1e-4,1e-1,10)),
    'l1_reg':[1e-4,0]
}
GLM = val_fun(GLMRegression,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld)

Model with params: {'knots': 3, 'l1_reg': 0.0001, 'lmd': 0.0001} finished.
with out-of-sample R squared on validation set: -3.05517%
************************************************************
Model with params: {'knots': 3, 'l1_reg': 0.0001, 'lmd': 0.1} finished.
with out-of-sample R squared on validation set: 0.00000%
************************************************************
Model with params: {'knots': 3, 'l1_reg': 0, 'lmd': 0.0001} finished.
with out-of-sample R squared on validation set: -3.08258%
************************************************************
Model with params: {'knots': 3, 'l1_reg': 0, 'lmd': 0.1} finished.
with out-of-sample R squared on validation set: 0.00000%
************************************************************

############################################################
Tuning process finished!!!
The best setting is: {'knots': 3, 'l1_reg': 0.0001, 'lmd': 0.1}
with R2oos 0.00% on validation set.
#####################################################

In [38]:
%%time
evaluate(y_trn, GLM.predict(X_trn), insample=True) 
evaluate(y_tst, GLM.predict(X_tst))

***************In-Sample Metrics***************
The in-sample R2 is -0.83%
The in-sample MSE is 0.028
***************Out-of-Sample Metrics***************
The out-of-sample R2 is 0.00%
The out-of-sample MSE is 0.026
CPU times: total: 1min 57s
Wall time: 1min 57s


In [39]:
output = pd.DataFrame(columns = ["OLS + H", "OLS-3 + H", "PCR", "PLS", "Enet + H", "GLM + H", "RF", "GBRT + H"], data=np.zeros((3,8)))
output.index = ["All", "Top 1000", "Bottom 1000"]
output.iloc[0,0] =r2_score(y_tst, OLS_H.predict(X_tst))
output.iloc[0,2] = r2_score(y_tst, PCR.predict(X_tst))
output.iloc[0,3] = r2_score(y_tst, PLS.predict(X_tst))
output.iloc[0,4] = r2_score(y_tst, EN_my_mse_rdm.predict(X_tst))
output.iloc[0,5] = r2_score(y_tst, GLM.predict(X_tst))

output.iloc[0,1] = r2_score(y_tst, OLS_3.predict(X_tst[features_3]))

## 1.2 Ensemble Models

### 1.2.0 Additional Functions

In [40]:
# huber loss function for customized objective function

# gradient of huber loss with respect to y_pred
def grad_huber_obj(y_true, y_pred):
    xi = 1.35 
    # Though I do not want to make it hard-coded, lightgbm, behind the scene, evaluates the # of parameters
    # of the objective function first, then pass according # of parameters. I tried to use partial to set 
    # the value of xi. It did not work.
    # I refer the readers to the source code to have a better understanding of the issue:
    # (https://github.com/microsoft/LightGBM/blob/master/python-package/lightgbm/sklearn.py)
    y_true, y_pred = np.array(y_true).flatten(), np.array(y_pred).flatten()
    N = len(y_true)
    resid = y_true - y_pred
    ind_m = np.where(np.abs(resid)<=xi)
    ind_u = np.where(resid>xi)
    ind_l = np.where(resid< -xi)
    grad = np.zeros(N)
    try:
        grad[ind_m] = (-2*(y_true-y_pred))[ind_m]
    except:
        pass
    try:
        grad[ind_u] = np.repeat(2*xi,N)[ind_u]
    except:
        pass
    try:
        grad[ind_l] = np.repeat(-2*xi,N)[ind_l]
    except:
        pass
    return grad/N

# hessian of huber loss with respect to y_pred
def hess_huber_obj(y_true, y_pred):
    xi = 1.35
    y_true, y_pred = np.array(y_true).flatten(), np.array(y_pred).flatten()
    N = len(y_true)
    resid = y_true - y_pred
    ind_m = np.where(np.abs(resid)<=xi)
    ind_u = np.where(resid>xi)
    ind_l = np.where(resid< -xi)
    hess = np.zeros(N)
    try:
        hess[ind_m] = np.repeat(2,N)[ind_m]
    except:
        pass
    return hess/N

# huber loss for lgbm
def huber_obj(y_true, y_pred):
    grad = grad_huber_obj(y_true, y_pred)
    hess = hess_huber_obj(y_true, y_pred)
    return grad, hess

### 1.2.1 Random Forest

In [41]:
params = {
    'n_estimators': [300],
    'max_depth': [3, 6],
    'max_features': [30, 50, 100],
    'random_state': [12308]
}
RF = val_fun(RandomForestRegressor,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld)

Model with params: {'max_depth': 3, 'max_features': 30, 'n_estimators': 300, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 0.31521%
************************************************************
Model with params: {'max_depth': 3, 'max_features': 50, 'n_estimators': 300, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 0.21073%
************************************************************
Model with params: {'max_depth': 3, 'max_features': 100, 'n_estimators': 300, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 0.33682%
************************************************************
Model with params: {'max_depth': 6, 'max_features': 30, 'n_estimators': 300, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 0.82757%
************************************************************
Model with params: {'max_depth': 6, 'max_features': 50, 'n_estimators': 300, 'random_st

In [42]:
%%time

evaluate(y_trn, RF.predict(X_trn), insample=True) 
evaluate(y_tst, RF.predict(X_tst))

***************In-Sample Metrics***************
The in-sample R2 is 15.23%
The in-sample MSE is 0.024
***************Out-of-Sample Metrics***************
The out-of-sample R2 is 1.46%
The out-of-sample MSE is 0.026
CPU times: total: 3.44 s
Wall time: 3.43 s


### 1.2.2 Gradient Boosting (XGBRegression)

In [43]:
params = {
    'objective':[None, huber_obj],
    'max_depth':[1,2],
    'n_estimators':[500, 800, 1000],
    'random_state':[12308],
    'learning_rate':[.01,.1]
}
LGBM = val_fun(XGBRegressor,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld)

Model with params: {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 500, 'objective': None, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 0.46913%
************************************************************
Model with params: {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 500, 'objective': <function huber_obj at 0x000001E787A3D040>, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: -2.41942%
************************************************************
Model with params: {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 800, 'objective': None, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 0.87788%
************************************************************
Model with params: {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 800, 'objective': <function huber_obj at 0x000001E787A3D040>, 'random_state': 12308} finished.
with out-of-sample R squared on validatio

In [44]:
%%time
evaluate(y_trn, LGBM.predict(X_trn), insample=True) 
evaluate(y_tst, LGBM.predict(X_tst))

***************In-Sample Metrics***************
The in-sample R2 is 40.88%
The in-sample MSE is 0.017
***************Out-of-Sample Metrics***************
The out-of-sample R2 is 14.99%
The out-of-sample MSE is 0.021
CPU times: total: 5.56 s
Wall time: 734 ms


In [45]:
output.iloc[0,6] = r2_score(y_tst, RF.predict(X_tst))
output.iloc[0,7] = r2_score(y_tst, LGBM.predict(X_tst))

## 1.3 Neural Networks

In [46]:
# customized metrics
# out-of-sample r squared for keras
def R_oos_tf(y_true, y_pred):
    resid = tf.square(y_true-y_pred)
    denom = tf.square(y_true)
    return 1 - tf.divide(tf.reduce_mean(resid),tf.reduce_mean(denom))

# data standardization
# please standardize the data if BatchNormalization is not used
def standardize(X_trn, X_vld, X_tst):
    mu_trn = np.mean(np.array(X_trn),axis=0).reshape((1,X_trn.shape[1]))
    sigma_trn = np.std(np.array(X_trn),axis=0).reshape((1,X_trn.shape[1]))

    X_trn_std = (np.array(X_trn)-mu_trn)/sigma_trn
    X_vld_std = (np.array(X_vld)-mu_trn)/sigma_trn
    X_tst_std = (np.array(X_tst)-mu_trn)/sigma_trn
    return X_trn_std, X_vld_std, X_tst_std

# NN class
class NN:
    
    def __init__(
        self, n_layers=1, loss='mse', l1=1e-5, l2=0, learning_rate=.01, BatchNormalization=True, patience=5,
        epochs=100, batch_size=3000, verbose=1, random_state=12308, monitor='val_R_oos_tf', base_neurons=5
    ):
        self.n_layers = n_layers
        self.l1 = l1
        self.l2 = l2
        self.learning_rate = learning_rate
        self.BatchNormalization = BatchNormalization
        self.patience = patience
        self.epochs = epochs
        self.batch_size = batch_size
        self.verbose = verbose
        self.random_state = random_state
        self.monitor = monitor
        self.base_neurons = base_neurons

    def set_params(self, **params):
        for param in params.keys():
            setattr(self, param, params[param])
        return self
    
    def fit(self, X_trn, y_trn, X_vld, y_vld):
        # fix random seed for reproducibility
        random.seed(self.random_state)
        np.random.seed(self.random_state)
        tf.random.set_seed(self.random_state)
        
        # model construction
        mod = Sequential()
        mod.add(Input(shape=(X_trn.shape[1],)))
        
        for i in np.arange(self.n_layers,0,-1):
            if self.n_layers>self.base_neurons:
                if self.n_layers == i:
                    mod.add(Dense(2**i, activation='relu'))
                else:
                    mod.add(Dense(2**i, activation='relu', kernel_regularizer=L1L2(self.l1,self.l2)))
            else:
                if self.n_layers == i:
                    mod.add(Dense(2**(self.base_neurons-(self.n_layers-i)), activation='relu'))
                else:
                    mod.add(Dense(2**(self.base_neurons-(self.n_layers-i)), 
                                  activation='relu', kernel_regularizer=L1L2(self.l1,self.l2)))
            if self.BatchNormalization:
                mod.add(BatchNormalization())
        
        mod.add(Dense(1, kernel_regularizer=L1L2(self.l1,self.l2)))
        
        # early stopping
        earlystop = tf.keras.callbacks.EarlyStopping(monitor=self.monitor, patience=self.patience, mode = 'max')

        # Adam solver
        opt = Adam(learning_rate=self.learning_rate)
        
        # compile the model
        mod.compile(loss=self.loss,
                    optimizer=opt,
                    metrics=[R_oos_tf])

        # fit the model
        mod.fit(X_trn, np.array(y_trn).reshape((len(y_trn),1)), epochs=self.epochs, batch_size=self.batch_size, 
                callbacks=[earlystop], verbose=self.verbose, 
                validation_data=(X_vld,np.array(y_vld).reshape((len(y_vld),1))))
        
        self.model = mod
        return self
    
    def predict(self, X):
        return self.model.predict(X, verbose=self.verbose)

In [47]:
%%time
# NN1-Regression-[32(relu)-1(linear)]

params = {
    'n_layers': [1],
    'loss': ['mse'],
    'l1': [1e-5,1e-3],
    'learning_rate': [.001,.01],
    'batch_size': [int(X_trn.shape[0]/50)],
    'epochs': [100],
    'random_state': [12308],
    'BatchNormalization': [True],
    'patience':[5],
    'verbose': [0],
    'monitor':['val_loss','val_R_oos_tf'],
}
NN1 = val_fun(NN,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,is_NN=True,sleep=5)

Model with params: {'BatchNormalization': True, 'batch_size': 2694, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.001, 'loss': 'mse', 'monitor': 'val_loss', 'n_layers': 1, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -0.01158%
************************************************************
Model with params: {'BatchNormalization': True, 'batch_size': 2694, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.001, 'loss': 'mse', 'monitor': 'val_R_oos_tf', 'n_layers': 1, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -1.28197%
************************************************************
Model with params: {'BatchNormalization': True, 'batch_size': 2694, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.01, 'loss': 'mse', 'monitor': 'val_loss', 'n_layers': 1, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -0.065

In [48]:
%%time
# NN2-Regression-[32(relu)-16(relu)-1(linear)]
params = {
    'n_layers': [2],
    'loss': ['mse'],
    'l1': [1e-5,1e-3],
    'learning_rate': [.001,.01],
    'batch_size': [int(X_trn.shape[0]/50)],
    'epochs': [100],
    'random_state': [12308],
    'BatchNormalization': [True],
    'patience':[5],
    'verbose': [0],
    'monitor':['val_loss','val_R_oos_tf']
}
NN2 = val_fun(NN,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,is_NN=True,sleep=5)

Model with params: {'BatchNormalization': True, 'batch_size': 2694, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.001, 'loss': 'mse', 'monitor': 'val_loss', 'n_layers': 2, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -220.98727%
************************************************************
Model with params: {'BatchNormalization': True, 'batch_size': 2694, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.001, 'loss': 'mse', 'monitor': 'val_R_oos_tf', 'n_layers': 2, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -4.04173%
************************************************************
Model with params: {'BatchNormalization': True, 'batch_size': 2694, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.01, 'loss': 'mse', 'monitor': 'val_loss', 'n_layers': 2, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -14.

In [49]:
%%time
# NN3-Regression-[32(relu)-16(relu)-8(relu)-1(linear)]
params = {
    'n_layers': [3],
    'loss': ['mse'],
    'l1': [1e-5,1e-3],
    'learning_rate': [.001,.01],
    'batch_size': [int(X_trn.shape[0]/50)],
    'epochs': [100],
    'random_state': [12308],
    'BatchNormalization': [True],
    'patience':[5],
    'verbose': [0],
    'monitor':['val_loss','val_R_oos_tf']
}
NN3 = val_fun(NN,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,is_NN=True,sleep=5)

Model with params: {'BatchNormalization': True, 'batch_size': 2694, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.001, 'loss': 'mse', 'monitor': 'val_loss', 'n_layers': 3, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -8.58787%
************************************************************
Model with params: {'BatchNormalization': True, 'batch_size': 2694, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.001, 'loss': 'mse', 'monitor': 'val_R_oos_tf', 'n_layers': 3, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -5.80852%
************************************************************
Model with params: {'BatchNormalization': True, 'batch_size': 2694, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.01, 'loss': 'mse', 'monitor': 'val_loss', 'n_layers': 3, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -1.183

In [50]:
%%time
# NN4-Regression-[32(relu)-16(relu)-8(relu)-4(relu)-1(linear)]
params = {
    'n_layers': [4],
    'loss': ['mse'],
    'l1': [1e-5,1e-3],
    'learning_rate': [.001,.01],
    'batch_size': [int(X_trn.shape[0]/50)],
    'epochs': [100],
    'random_state': [12308],
    'BatchNormalization': [True],
    'patience':[5],
    'verbose': [0],
    'monitor':['val_loss','val_R_oos_tf']
}
NN4 = val_fun(NN,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,is_NN=True,sleep=5)

Model with params: {'BatchNormalization': True, 'batch_size': 2694, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.001, 'loss': 'mse', 'monitor': 'val_loss', 'n_layers': 4, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -17.68690%
************************************************************
Model with params: {'BatchNormalization': True, 'batch_size': 2694, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.001, 'loss': 'mse', 'monitor': 'val_R_oos_tf', 'n_layers': 4, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -10.78903%
************************************************************
Model with params: {'BatchNormalization': True, 'batch_size': 2694, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.01, 'loss': 'mse', 'monitor': 'val_loss', 'n_layers': 4, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -1.6

In [51]:
%%time
# NN5-Regression-[32(relu)-16(relu)-8(relu)-4(relu)-2(relu)-1(linear)]
params = {
    'n_layers': [5],
    'loss': ['mse'],
    'l1': [1e-5,1e-3],
    'learning_rate': [.001,.01],
    'batch_size': [int(X_trn.shape[0]/50)],
    'epochs': [100],
    'random_state': [12308],
    'BatchNormalization': [True],
    'patience':[5],
    'verbose': [0],
    'monitor':['val_loss','val_R_oos_tf']
}
NN5 = val_fun(NN,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,is_NN=True,sleep=5)

Model with params: {'BatchNormalization': True, 'batch_size': 2694, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.001, 'loss': 'mse', 'monitor': 'val_loss', 'n_layers': 5, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -0.67422%
************************************************************
Model with params: {'BatchNormalization': True, 'batch_size': 2694, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.001, 'loss': 'mse', 'monitor': 'val_R_oos_tf', 'n_layers': 5, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -1.03517%
************************************************************
Model with params: {'BatchNormalization': True, 'batch_size': 2694, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.01, 'loss': 'mse', 'monitor': 'val_loss', 'n_layers': 5, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -1.130

In [55]:
output['NN1'] = np.array([0,0,0])
output['NN2'] = np.array([0,0,0])
output['NN3'] = np.array([0,0,0])
output['NN4'] = np.array([0,0,0])
output['NN5'] = np.array([0,0,0])
output.iloc[0,8] = r2_score(y_tst, NN1.predict(X_tst))
output.iloc[0,9] = r2_score(y_tst, NN2.predict(X_tst))
output.iloc[0,10] = r2_score(y_tst, NN3.predict(X_tst))
output.iloc[0,11] = r2_score(y_tst, NN4.predict(X_tst))
output.iloc[0,12] = r2_score(y_tst, NN5.predict(X_tst))

## 1.3 Top

In [56]:
b1_top = b1_d.sort_values('mvel1',ascending=False).groupby('date').head(1000)

In [57]:
index_co = b1_top.index
features = list(set(b1_top.columns).difference({'permno','DATE','RET'}))
X = MinMaxScaler((-1,1)).fit_transform(b1_top[features])
X = pd.DataFrame(X, columns=features, index=index_co)
y = b1_top['RET']

stdt_vld = starting + (ending - starting)/3
stdt_tst = starting + 2*(ending - starting)/3

X_trn = X[X.index.get_level_values(level = "date") <= stdt_vld]
y_trn = y[y.index.get_level_values(level = "date") <= stdt_vld]

X_vld = X[(X.index.get_level_values(level = "date") >= stdt_vld) & (X.index.get_level_values(level = "date") <= stdt_tst)]
y_vld = y[(y.index.get_level_values(level = "date") >= stdt_vld) & (y.index.get_level_values(level = "date") <= stdt_tst)]

X_tst = X[X.index.get_level_values(level = "date") >= stdt_tst]
y_tst = y[y.index.get_level_values(level = "date") >= stdt_tst]

In [58]:
epsilon = np.max(((y_trn-OLS.predict(X_trn)).quantile(.999),1))
OLS_H = HuberRegressor(epsilon=epsilon).fit(X_trn,y_trn)

params = {'n_components': [1, 5, 10, 50]}
PLS = val_fun(PLSRegression,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld)

params = {'n_PCs':[1,3,5,7,10,50],'loss':['mse','huber']}
PCR = val_fun(PCRegressor,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,sleep=3)

EN_my_mse_rdm = ENet(lmd=.01,rho=.5,loss='huber').fit(X_trn,y_trn)

params = {
    'knots':[3],
    'lmd':[1e-4,1e-1],#list(np.linspace(1e-4,1e-1,10)),
    'l1_reg':[1e-4,0]
}
GLM = val_fun(GLMRegression,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld)

features_3 = ['mvel1','bm','mom1m','mom6m','mom12m','mom36m']
OLS_3 = LinearRegression().fit(X_trn[features_3],y_trn)

Model with params: {'n_components': 1} finished.
with out-of-sample R squared on validation set: -0.51006%
************************************************************
Model with params: {'n_components': 5} finished.
with out-of-sample R squared on validation set: -10.54365%
************************************************************
Model with params: {'n_components': 10} finished.
with out-of-sample R squared on validation set: -13.31591%
************************************************************
Model with params: {'n_components': 50} finished.
with out-of-sample R squared on validation set: -12.98592%
************************************************************

############################################################
Tuning process finished!!!
The best setting is: {'n_components': 1}
with R2oos -0.51% on validation set.
############################################################
Model with params: {'loss': 'mse', 'n_PCs': 1} finished.
with out-of-sample R squared on valida

In [59]:
output.iloc[1,0] =r2_score(y_tst, OLS_H.predict(X_tst))
output.iloc[1,2] = r2_score(y_tst, PCR.predict(X_tst))
output.iloc[1,3] = r2_score(y_tst, PLS.predict(X_tst))
output.iloc[1,4] = r2_score(y_tst, EN_my_mse_rdm.predict(X_tst))
output.iloc[1,5] = r2_score(y_tst, GLM.predict(X_tst))

output.iloc[1,1] = r2_score(y_tst, OLS_3.predict(X_tst[features_3]))

In [60]:
params = {
    'n_estimators': [300],
    'max_depth': [3, 6],
    'max_features': [30, 50, 100],
    'random_state': [12308]
}
RF = val_fun(RandomForestRegressor,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld)

Model with params: {'max_depth': 3, 'max_features': 30, 'n_estimators': 300, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 0.85469%
************************************************************
Model with params: {'max_depth': 3, 'max_features': 50, 'n_estimators': 300, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 1.25279%
************************************************************
Model with params: {'max_depth': 3, 'max_features': 100, 'n_estimators': 300, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 0.62587%
************************************************************
Model with params: {'max_depth': 6, 'max_features': 30, 'n_estimators': 300, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 3.00380%
************************************************************
Model with params: {'max_depth': 6, 'max_features': 50, 'n_estimators': 300, 'random_st

In [61]:
params = {
    'objective':[None, huber_obj],
    'max_depth':[1,2],
    'n_estimators':[500, 800, 1000],
    'random_state':[12308],
    'learning_rate':[.01,.1]
}
LGBM = val_fun(XGBRegressor,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld)

Model with params: {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 500, 'objective': None, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 1.00731%
************************************************************
Model with params: {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 500, 'objective': <function huber_obj at 0x000001E787A3D040>, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: -5.69424%
************************************************************
Model with params: {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 800, 'objective': None, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 1.12796%
************************************************************
Model with params: {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 800, 'objective': <function huber_obj at 0x000001E787A3D040>, 'random_state': 12308} finished.
with out-of-sample R squared on validatio

In [62]:
output.iloc[1,6] = r2_score(y_tst, RF.predict(X_tst))
output.iloc[1,7] = r2_score(y_tst, LGBM.predict(X_tst))

In [63]:
%%time
# NN1-Regression-[32(relu)-1(linear)]

params = {
    'n_layers': [1],
    'loss': ['mse'],
    'l1': [1e-5,1e-3],
    'learning_rate': [.001,.01],
    'batch_size': [int(X_trn.shape[0]/50)],
    'epochs': [100],
    'random_state': [12308],
    'BatchNormalization': [True],
    'patience':[5],
    'verbose': [0],
    'monitor':['val_loss','val_R_oos_tf'],
}
NN1 = val_fun(NN,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,is_NN=True,sleep=5)

%%time
# NN2-Regression-[32(relu)-16(relu)-1(linear)]
params = {
    'n_layers': [2],
    'loss': ['mse'],
    'l1': [1e-5,1e-3],
    'learning_rate': [.001,.01],
    'batch_size': [int(X_trn.shape[0]/50)],
    'epochs': [100],
    'random_state': [12308],
    'BatchNormalization': [True],
    'patience':[5],
    'verbose': [0],
    'monitor':['val_loss','val_R_oos_tf']
}
NN2 = val_fun(NN,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,is_NN=True,sleep=5)

%%time
# NN3-Regression-[32(relu)-16(relu)-8(relu)-1(linear)]
params = {
    'n_layers': [3],
    'loss': ['mse'],
    'l1': [1e-5,1e-3],
    'learning_rate': [.001,.01],
    'batch_size': [int(X_trn.shape[0]/50)],
    'epochs': [100],
    'random_state': [12308],
    'BatchNormalization': [True],
    'patience':[5],
    'verbose': [0],
    'monitor':['val_loss','val_R_oos_tf']
}
NN3 = val_fun(NN,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,is_NN=True,sleep=5)

%%time
# NN4-Regression-[32(relu)-16(relu)-8(relu)-4(relu)-1(linear)]
params = {
    'n_layers': [4],
    'loss': ['mse'],
    'l1': [1e-5,1e-3],
    'learning_rate': [.001,.01],
    'batch_size': [int(X_trn.shape[0]/50)],
    'epochs': [100],
    'random_state': [12308],
    'BatchNormalization': [True],
    'patience':[5],
    'verbose': [0],
    'monitor':['val_loss','val_R_oos_tf']
}
NN4 = val_fun(NN,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,is_NN=True,sleep=5)

%%time
# NN5-Regression-[32(relu)-16(relu)-8(relu)-4(relu)-2(relu)-1(linear)]
params = {
    'n_layers': [5],
    'loss': ['mse'],
    'l1': [1e-5,1e-3],
    'learning_rate': [.001,.01],
    'batch_size': [int(X_trn.shape[0]/50)],
    'epochs': [100],
    'random_state': [12308],
    'BatchNormalization': [True],
    'patience':[5],
    'verbose': [0],
    'monitor':['val_loss','val_R_oos_tf']
}
NN5 = val_fun(NN,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,is_NN=True,sleep=5)

Model with params: {'BatchNormalization': True, 'batch_size': 500, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.001, 'loss': 'mse', 'monitor': 'val_loss', 'n_layers': 1, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -0.40921%
************************************************************
Model with params: {'BatchNormalization': True, 'batch_size': 500, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.001, 'loss': 'mse', 'monitor': 'val_R_oos_tf', 'n_layers': 1, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -3.43583%
************************************************************
Model with params: {'BatchNormalization': True, 'batch_size': 500, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.01, 'loss': 'mse', 'monitor': 'val_loss', 'n_layers': 1, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -0.01586%

UsageError: Line magic function `%%time` not found.


In [64]:
output.iloc[1,8] = r2_score(y_tst, NN1.predict(X_tst))
output.iloc[1,9] = r2_score(y_tst, NN2.predict(X_tst))
output.iloc[1,10] = r2_score(y_tst, NN3.predict(X_tst))
output.iloc[1,11] = r2_score(y_tst, NN4.predict(X_tst))
output.iloc[1,12] = r2_score(y_tst, NN5.predict(X_tst))

## 1.5 Bottom

In [65]:
b1_bot = b1_d.sort_values('mvel1',ascending=False).groupby('date').tail(1000)

In [66]:
index_co = b1_bot.index
features = list(set(b1_bot.columns).difference({'permno','DATE','RET'}))
X = MinMaxScaler((-1,1)).fit_transform(b1_bot[features])
X = pd.DataFrame(X, columns=features, index=index_co)
y = b1_bot['RET']

stdt_vld = starting + (ending - starting)/3
stdt_tst = starting + 2*(ending - starting)/3

X_trn = X[X.index.get_level_values(level = "date") <= stdt_vld]
y_trn = y[y.index.get_level_values(level = "date") <= stdt_vld]

X_vld = X[(X.index.get_level_values(level = "date") >= stdt_vld) & (X.index.get_level_values(level = "date") <= stdt_tst)]
y_vld = y[(y.index.get_level_values(level = "date") >= stdt_vld) & (y.index.get_level_values(level = "date") <= stdt_tst)]

X_tst = X[X.index.get_level_values(level = "date") >= stdt_tst]
y_tst = y[y.index.get_level_values(level = "date") >= stdt_tst]

In [67]:
epsilon = np.max(((y_trn-OLS.predict(X_trn)).quantile(.999),1))
OLS_H = HuberRegressor(epsilon=epsilon).fit(X_trn,y_trn)

params = {'n_components': [1, 5, 10, 50]}
PLS = val_fun(PLSRegression,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld)

params = {'n_PCs':[1,3,5,7,10,50],'loss':['mse','huber']}
PCR = val_fun(PCRegressor,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,sleep=3)

EN_my_mse_rdm = ENet(lmd=.01,rho=.5,loss='huber').fit(X_trn,y_trn)

params = {
    'knots':[3],
    'lmd':[1e-4,1e-1],#list(np.linspace(1e-4,1e-1,10)),
    'l1_reg':[1e-4,0]
}
GLM = val_fun(GLMRegression,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld)

features_3 = ['mvel1','bm','mom1m','mom6m','mom12m','mom36m']
OLS_3 = LinearRegression().fit(X_trn[features_3],y_trn)

Model with params: {'n_components': 1} finished.
with out-of-sample R squared on validation set: -4.94598%
************************************************************
Model with params: {'n_components': 5} finished.
with out-of-sample R squared on validation set: -2.12986%
************************************************************
Model with params: {'n_components': 10} finished.
with out-of-sample R squared on validation set: -2.96738%
************************************************************
Model with params: {'n_components': 50} finished.
with out-of-sample R squared on validation set: -3.07035%
************************************************************

############################################################
Tuning process finished!!!
The best setting is: {'n_components': 5}
with R2oos -2.13% on validation set.
############################################################
Model with params: {'loss': 'mse', 'n_PCs': 1} finished.
with out-of-sample R squared on validatio

In [68]:
output.iloc[2,0] =r2_score(y_tst, OLS_H.predict(X_tst))
output.iloc[2,2] = r2_score(y_tst, PCR.predict(X_tst))
output.iloc[2,3] = r2_score(y_tst, PLS.predict(X_tst))
output.iloc[2,4] = r2_score(y_tst, EN_my_mse_rdm.predict(X_tst))
output.iloc[2,5] = r2_score(y_tst, GLM.predict(X_tst))

output.iloc[2,1] = r2_score(y_tst, OLS_3.predict(X_tst[features_3]))

In [69]:
params = {
    'n_estimators': [300],
    'max_depth': [3, 6],
    'max_features': [30, 50, 100],
    'random_state': [12308]
}
RF = val_fun(RandomForestRegressor,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld)

Model with params: {'max_depth': 3, 'max_features': 30, 'n_estimators': 300, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 1.50883%
************************************************************
Model with params: {'max_depth': 3, 'max_features': 50, 'n_estimators': 300, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 1.51341%
************************************************************
Model with params: {'max_depth': 3, 'max_features': 100, 'n_estimators': 300, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 1.10074%
************************************************************
Model with params: {'max_depth': 6, 'max_features': 30, 'n_estimators': 300, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 3.46021%
************************************************************
Model with params: {'max_depth': 6, 'max_features': 50, 'n_estimators': 300, 'random_st

In [70]:
params = {
    'objective':[None, huber_obj],
    'max_depth':[1,2],
    'n_estimators':[500, 800, 1000],
    'random_state':[12308],
    'learning_rate':[.01,.1]
}
LGBM = val_fun(XGBRegressor,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld)

Model with params: {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 500, 'objective': None, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 2.53077%
************************************************************
Model with params: {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 500, 'objective': <function huber_obj at 0x000001E787A3D040>, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: -0.80458%
************************************************************
Model with params: {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 800, 'objective': None, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 3.36414%
************************************************************
Model with params: {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 800, 'objective': <function huber_obj at 0x000001E787A3D040>, 'random_state': 12308} finished.
with out-of-sample R squared on validatio

In [71]:
output.iloc[2,6] = r2_score(y_tst, RF.predict(X_tst))
output.iloc[2,7] = r2_score(y_tst, LGBM.predict(X_tst))

In [72]:
%%time
# NN1-Regression-[32(relu)-1(linear)]

params = {
    'n_layers': [1],
    'loss': ['mse'],
    'l1': [1e-5,1e-3],
    'learning_rate': [.001,.01],
    'batch_size': [int(X_trn.shape[0]/50)],
    'epochs': [100],
    'random_state': [12308],
    'BatchNormalization': [True],
    'patience':[5],
    'verbose': [0],
    'monitor':['val_loss','val_R_oos_tf'],
}
NN1 = val_fun(NN,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,is_NN=True,sleep=5)

%%time
# NN2-Regression-[32(relu)-16(relu)-1(linear)]
params = {
    'n_layers': [2],
    'loss': ['mse'],
    'l1': [1e-5,1e-3],
    'learning_rate': [.001,.01],
    'batch_size': [int(X_trn.shape[0]/50)],
    'epochs': [100],
    'random_state': [12308],
    'BatchNormalization': [True],
    'patience':[5],
    'verbose': [0],
    'monitor':['val_loss','val_R_oos_tf']
}
NN2 = val_fun(NN,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,is_NN=True,sleep=5)

%%time
# NN3-Regression-[32(relu)-16(relu)-8(relu)-1(linear)]
params = {
    'n_layers': [3],
    'loss': ['mse'],
    'l1': [1e-5,1e-3],
    'learning_rate': [.001,.01],
    'batch_size': [int(X_trn.shape[0]/50)],
    'epochs': [100],
    'random_state': [12308],
    'BatchNormalization': [True],
    'patience':[5],
    'verbose': [0],
    'monitor':['val_loss','val_R_oos_tf']
}
NN3 = val_fun(NN,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,is_NN=True,sleep=5)

%%time
# NN4-Regression-[32(relu)-16(relu)-8(relu)-4(relu)-1(linear)]
params = {
    'n_layers': [4],
    'loss': ['mse'],
    'l1': [1e-5,1e-3],
    'learning_rate': [.001,.01],
    'batch_size': [int(X_trn.shape[0]/50)],
    'epochs': [100],
    'random_state': [12308],
    'BatchNormalization': [True],
    'patience':[5],
    'verbose': [0],
    'monitor':['val_loss','val_R_oos_tf']
}
NN4 = val_fun(NN,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,is_NN=True,sleep=5)

%%time
# NN5-Regression-[32(relu)-16(relu)-8(relu)-4(relu)-2(relu)-1(linear)]
params = {
    'n_layers': [5],
    'loss': ['mse'],
    'l1': [1e-5,1e-3],
    'learning_rate': [.001,.01],
    'batch_size': [int(X_trn.shape[0]/50)],
    'epochs': [100],
    'random_state': [12308],
    'BatchNormalization': [True],
    'patience':[5],
    'verbose': [0],
    'monitor':['val_loss','val_R_oos_tf']
}
NN5 = val_fun(NN,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,is_NN=True,sleep=5)

Model with params: {'BatchNormalization': True, 'batch_size': 500, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.001, 'loss': 'mse', 'monitor': 'val_loss', 'n_layers': 1, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -0.00355%
************************************************************
Model with params: {'BatchNormalization': True, 'batch_size': 500, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.001, 'loss': 'mse', 'monitor': 'val_R_oos_tf', 'n_layers': 1, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: 0.48623%
************************************************************
Model with params: {'BatchNormalization': True, 'batch_size': 500, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.01, 'loss': 'mse', 'monitor': 'val_loss', 'n_layers': 1, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: 0.33061%
*

UsageError: Line magic function `%%time` not found.


In [73]:
output.iloc[2,8] = r2_score(y_tst, NN1.predict(X_tst))
output.iloc[2,9] = r2_score(y_tst, NN2.predict(X_tst))
output.iloc[2,10] = r2_score(y_tst, NN3.predict(X_tst))
output.iloc[2,11] = r2_score(y_tst, NN4.predict(X_tst))
output.iloc[2,12] = r2_score(y_tst, NN5.predict(X_tst))

In [74]:
output.to_excel("reality_here.xlsx")

In [75]:
output

,OLS + H,OLS-3 + H,PCR,PLS,Enet + H,GLM + H,RF,GBRT + H,NN1,NN2,NN3,NN4,NN5
All,-0.033009,-0.009846,-0.021733,-0.030201,-0.005345,-0.002751,0.014888,0.191139,-2.286733,-0.131319,-0.007220,-0.005772,-0.000042
Top 1000,-0.055939,-0.010582,-0.017403,-0.059468,-0.003677,-0.041385,0.026023,0.239203,-0.197176,-1.598856,-0.050293,-0.001856,-0.008123
Bottom 1000,-0.025572,-0.009002,-0.012734,-0.201587,-0.004904,-0.001606,0.064284,0.263792,-0.204276,-0.801802,-0.002966,-0.001504,-0.000022


-------------------

-------------------------

### Old Code (Do not run it)

In [ ]:
MC = 1  # setup "Monte-Carlo" number
path = './Simu'  # set your own folder path 
datanum = 50 #Number of characteristics in the data (the only option is 50)
dirstock = f"{path}/Simu_p{datanum}/"

hh = [1] # Calculating monthly returns

# hh = [1, 3, 6, 12]  # correspond to monthly, quarterly, half-year, and annually returns (have to but it into loop)

title = f"{path}/Simu_p{datanum}/Reg{hh[0]}"

if not os.path.exists(title) and MC == 1:
    os.makedirs(title)
titleB = f"{title}/B"
if not os.path.exists(titleB) and MC == 1:
    os.makedirs(titleB)
if datanum == 50:
    nump = 50

M = 1 #Model number
mu = 0.2 * np.sqrt(hh[0])
tol = 10**(-10)
mo = 1 
nump = 50
N = 200   # Number of CS tickers
m = nump * 2   # Number of Characteristics
T = 180   # Number of Time Periods

per = np.tile(np.arange(1, N+1), T)
time = np.repeat(np.arange(1, T+1), N)
stdv = 0.05
theta_w = 0.005

# Read Files
path1 = dirstock + 'c' + str(M) + '_' + str(M) + '.csv'
path2 = dirstock + 'r' + str(mo) + '_' + str(M) + '.csv'
c = np.genfromtxt(path1, delimiter=',')
r1 = np.genfromtxt(path2, delimiter=',')

# Add Some Elements
daylen = np.repeat(N, T//3)
daylen_test = daylen.copy()
ind = np.arange(0, N*T//3)
xtrain = c[ind]
ytrain = r1[ind]
trainper = per[ind]
start_idx = math.floor(N * T / 3) + 1
end_idx = math.floor(N * (T * 2 / 3 - hh[0] + 1))
ind = list(range(start_idx, end_idx))
xtest = c[ind]
ytest = r1[ind]
testper = per[ind]

l1 = c.shape[0]
l2 = len(r1)
l3 = l2 - np.isnan(r1).sum()

ind = np.arange(N*(2*T//3), min([l1, l2, l3]))
xoos = c[ind]
yoos = r1[ind]

# Monthly Demean
ytrain_demean = ytrain - np.mean(ytrain)
ytest_demean = ytest - np.mean(ytest)
mtrain = np.mean(ytrain)
mtest = np.mean(ytest)

# Calculate Sufficient Stats
sd = np.zeros(xtrain.shape[1])
for i in range(xtrain.shape[1]):
    s = np.std(xtrain[:, i])
    if s > 0:
        xtrain[:, i] /= s
        xtest[:, i] /= s
        xoos[:, i] /= s
        sd[i] = s

XX = np.dot(xtrain.T, xtrain)
U, S, V = np.linalg.svd(XX)
L = S[0]
Y = ytrain_demean
XY = np.dot(xtrain.T, Y)

In [ ]:
#OLS

# Initialize arrays
r2_oos = np.zeros(13)
r2_is = np.zeros(13)
modeln = 0
groups = 0
nc = 0

# OLS+H
modeln += 1

clf = LinearRegression(fit_intercept=False)
clf.fit(xtrain, ytrain_demean)
b = clf.coef_

func = soft_threshodl
b = proximalH(groups, nc, xtest, mtrain, ytest, b, xtrain, ytrain_demean, mu, tol, L, 0, func)
yhatbig1 = np.dot(xoos, b) + mtrain
r2_oos[modeln - 1] = 1 - np.sum(np.power(yhatbig1 - yoos, 2)) / np.sum(np.power(yoos - mtrain, 2))
yhatbig1 = np.dot(xtrain, b) + mtrain
r2_is[modeln - 1] = 1 - np.sum(np.power(yhatbig1 - ytrain, 2)) / np.sum(np.power(ytrain - mtrain, 2))
pathb = f"{title}/B/b{mo}_{M}_{modeln}.csv"
pd.DataFrame(b).to_csv(pathb, index=False, header=False)
print(f"Simple OLS+H R2: {r2_oos[modeln - 1]:.3f}")

# PCR
modeln += 1
ne = 30
X = np.dot(xtrain.T, xtrain)
pca_vec = V.T
p1 = pca_vec[:, :ne]
Z = np.dot(xtrain, p1)
r = np.zeros((3, ne))
B = np.zeros((xtrain.shape[1], ne))
Y = ytrain_demean

for j in range(ne - 1):
    xx = Z[:, :j + 1]
    b = np.dot(np.linalg.inv(np.dot(xx.T, xx)), np.dot(xx.T, Y))
    b = np.dot(p1[:, :j + 1], b)

    yhatbig1 = np.dot(xtest, b) + mtrain
    r[0, j] = 1 - np.sum(np.power(yhatbig1 - ytest, 2)) / np.sum(np.power(ytest - mtrain, 2))
    yhatbig1 = np.dot(xoos, b) + mtrain
    r[1, j] = 1 - np.sum(np.power(yhatbig1 - yoos, 2)) / np.sum(np.power(yoos - mtrain, 2))
    yhatbig1 = np.dot(xtrain, b) + mtrain
    r[2, j] = 1 - np.sum(np.power(yhatbig1 - ytrain, 2)) / np.sum(np.power(ytrain - mtrain, 2))
    B[:, j] = b

b = np.zeros(xtest.shape[1])
j = ne - 1
yhatbig1 = np.dot(xtest, b) + mtrain
r[0, j] = 1 - np.sum(np.power(yhatbig1 - ytest, 2)) / np.sum(np.power(ytest - mtrain, 2))
yhatbig1 = np.dot(xoos, b) + mtrain
r[1, j] = 1 - np.sum(np.power(yhatbig1 - yoos, 2)) / np.sum(np.power(yoos - mtrain, 2))
yhatbig1 = np.dot(xtrain, b) + mtrain
r[2, j] = 1 - np.sum(np.power(yhatbig1 - ytrain, 2)) / np.sum(np.power(ytrain - mtrain, 2))
B[:, j] = b

r2_oos[modeln - 1] = r[1, fw1(r[0, :].tolist())]
r2_is[modeln - 1] = r[2, fw1(r[0, :].tolist())]
b = B[:, fw1(r[0, :].tolist())]
pathb = f"{title}/B/b{mo}_{M}_{modeln}.csv"
pd.DataFrame(b).to_csv(pathb, index=False, header=False)
print(f"PCR R2: {r2_oos[modeln - 1]:.3f}")

# PLS
modeln += 1
B = pls(xtrain, ytrain_demean, 30)
ne = 30
r = np.zeros((3, ne))
Y = ytrain_demean

for j in range(ne):
    b = B[:, j]
    yhatbig1 = np.dot(xtest, b) + mtrain
    r[0, j] = 1 - np.sum(np.power(yhatbig1 - ytest, 2)) / np.sum(np.power(ytest - mtrain, 2))
    yhatbig1 = np.dot(xoos, b) + mtrain
    r[1, j] = 1 - np.sum(np.power(yhatbig1 - yoos, 2)) / np.sum(np.power(yoos - mtrain, 2))
    yhatbig1 = np.dot(xtrain, b) + mtrain
    r[2, j] = 1 - np.sum(np.power(yhatbig1 - ytrain, 2)) / np.sum(np.power(ytrain - mtrain, 2))

r2_oos[modeln - 1] = r[1, fw1(r[0, :].tolist())]
r2_is[modeln - 1] = r[2, fw1(r[0, :].tolist())]
b = B[:, fw1(r[0, :].tolist())]
pathb = f"{title}/B/b{mo}_{M}_{modeln}.csv"
pd.DataFrame(b).to_csv(pathb, index=False, header=False)
print(f"PLS R2: {r2_oos[modeln - 1]:.3f}")


# Elastic Net
modeln += 1
lamv = np.arange(-2, 4.1, 0.1)
alpha = 0.5
r = np.zeros((3, len(lamv)))

for j in range(len(lamv)):
    l2 = 10 ** lamv[j]
    func = soft_threshode
    b = proximal(groups, nc, XX, XY, tol, L, l2, func)
    yhatbig1 = np.dot(xtest, b) + mtrain
    r[0, j] = 1 - np.sum(np.power(yhatbig1 - ytest, 2)) / np.sum(np.power(ytest - mtrain, 2))
    yhatbig1 = np.dot(xoos, b) + mtrain
    r[1, j] = 1 - np.sum(np.power(yhatbig1 - yoos, 2)) / np.sum(np.power(yoos - mtrain, 2))
    yhatbig1 = np.dot(xtrain, b) + mtrain
    r[2, j] = 1 - np.sum(np.power(yhatbig1 - ytrain, 2)) / np.sum(np.power(ytrain - mtrain, 2))

r2_oos[modeln - 1] = r[1, fw1(r[0, :].tolist())]
r2_is[modeln - 1] = r[2, fw1(r[0, :].tolist())]
l2 = 10 ** lamv[int(fw1(r[0]))]
func = soft_threshode
b = proximal(groups, nc, XX, XY, tol, L, l2, func)
pathb = f"{title}/B/b{mo}_{M}_{modeln}.csv"
np.savetxt(pathb, b, delimiter=',')
print('Enet R2 :', np.round(r2_oos[modeln-1], 3))

modeln += 1
func = soft_threshode
b = proximalH(groups, nc, xtest, mtrain, ytest, b, xtrain, ytrain_demean, mu, tol, L, l2, func)
yhatbig1 = np.dot(xoos, b) + mtrain
r2_oos[modeln-1] = 1 - np.sum(np.power(yhatbig1 - yoos, 2)) / np.sum(np.power(yoos - mtrain, 2))
yhatbig1 = np.dot(xtrain, b) + mtrain
r2_is[modeln-1] = 1 - np.sum(np.power(yhatbig1 - ytrain, 2)) / np.sum(np.power(ytrain - mtrain, 2))
pathb = f"{title}/B/b{mo}_{M}_{modeln}.csv"
np.savetxt(pathb, b, delimiter=',')
print('Enet+H R2 :', np.round(r2_oos[modeln-1], 3))

# Group Lasso
kn = 4
th = np.zeros((kn, xtrain.shape[1]))
th[1, :] = 0

for i in range(xtrain.shape[1]):
    th[:, i] = np.quantile(xtrain[:, i], np.arange(kn) / kn)

xtrain = cut_knots_degree2(xtrain, kn, th)
xtest = cut_knots_degree2(xtest, kn, th)
xoos = cut_knots_degree2(xoos, kn, th)

for i in range(xtrain.shape[1]):
    s = np.std(xtrain[:, i])
    if s > 0:
        xtrain[:, i] = xtrain[:, i] / s
        xtest[:, i] = xtest[:, i] / s
        xoos[:, i] = xoos[:, i] / s

Y = ytrain_demean
XX = xtrain.T @ xtrain
U, S, V = np.linalg.svd(XX)
L = S[0]
XY = xtrain.T @ Y

modeln += 1
lamv = np.arange(0.5, 3.1, 0.1)
nc = XX.shape[1] // (kn + 1)
groups = np.repeat(np.arange(1, nc + 1), kn + 1)
r = np.zeros((3, len(lamv)))

for j, lam in enumerate(lamv):
    l2 = 10 ** lam
    func = soft_threshodg
    b = proximal(groups, nc, XX, XY, tol, L, l2, func)
    yhatbig1 = xtest @ b + mtrain
    r[0, j] = 1 - np.sum((yhatbig1 - ytest) ** 2) / np.sum((ytest - mtrain) ** 2)
    yhatbig1 = xoos @ b + mtrain
    r[1, j] = 1 - np.sum((yhatbig1 - yoos) ** 2) / np.sum((yoos - mtrain) ** 2)
    yhatbig1 = xtrain @ b + mtrain
    r[2, j] = 1 - np.sum((yhatbig1 - ytrain) ** 2) / np.sum((ytrain - mtrain) ** 2)

r2_oos[modeln] = r[1, np.int16(fw1(r[0, :]))]
r2_is[modeln] = r[2, np.int16(fw1(r[0, :]))]
l2 = 10 ** lamv[np.int16(fw1(r[0, :]))]

func = soft_threshodg
b = proximal(groups, nc, XX, XY, tol, L, l2, func)
pathb = f"{title}/B/b"
pathb = f"{pathb}_{mo}_{M}_{modeln}.csv"
np.savetxt(pathb, b, delimiter=",")

print("Group Lasso R2:", np.round(r2_oos[modeln], 3))

modeln += 1
func = soft_threshodg
b = proximalH(groups, nc, xtest, mtrain, ytest, b, xtrain, ytrain_demean, mu, tol, L, l2, func)
yhatbig1 = xoos @ b + mtrain
r2_oos[modeln] = 1 - np.sum((yhatbig1 - yoos) ** 2) / np.sum((yoos - mtrain) ** 2)
yhatbig1 = xtrain @ b + mtrain
r2_is[modeln] = 1 - np.sum((yhatbig1 - ytrain) ** 2) / np.sum((ytrain - mtrain) ** 2)
pathb = f"{title}/B/b"
pathb = f"{pathb}_{mo}_{M}_{modeln}.csv"
np.savetxt(pathb, b, delimiter=",")

print("Group Lasso+H R2:", np.round(r2_oos[modeln], 3))

pathr = f"{title}/roos"
pathr = f"{pathr}_{mo}_{M}.csv"
np.savetxt(pathr, r2_oos, delimiter=",")

pathr = f"{title}/ris"
pathr = f"{pathr}_{mo}_{M}.csv"
np.savetxt(pathr, r2_is, delimiter=",")

# Ensemble Models

In [ ]:
MC = 1  # setup MC number
datanum = 50  # Or datanum = 100; separately run two cases
path = './Simu'  # set your own folder path
dirstock = os.path.join(path, f'SimuData_p{datanum}/')

for hh in [1]:
    # for hh in [1, 3, 6, 12]:  # correspond to monthly quarterly halfyear and annually returns
    title = os.path.join(path, f'Simu_p{datanum}/Tree{hh}')

    if not os.path.isdir(title) and MC == 1:
        os.makedirs(title)

    titleB = os.path.join(title, 'B')
    if not os.path.isdir(titleB) and MC == 1:
        os.makedirs(titleB)

    if datanum == 50:
        nump = 50
    if datanum == 100:
        nump = 100
        
        
    for M in [MC]:
        for mo in [1, 2]:
            print(f"### MCMC : {M}, Model : {mo} ###")
            N = 200  # Number of CS tickers
            m = nump * 2  # Number of Characteristics
            T = 180  # Number of Time Periods

            per = np.tile(np.arange(1, N + 1), T)
            time = np.repeat(np.arange(1, T + 1), N)
            stdv = 0.05
            theta_w = 0.005

            # Read Files
            path1 = f"{dirstock}c{M}.csv"
            path2 = f"{dirstock}r{mo}_{M}.csv"
            c = np.genfromtxt(path1, delimiter=',')
            r1 = np.genfromtxt(path2, delimiter=',')

            # Add Some Elements
            daylen = np.tile(N, T // 3)
            daylen_test = daylen
            ind = np.arange(0, int(N * T / 3))
            xtrain = c[ind, :]
            ytrain = r1[ind]
            trainper = per[ind]
            ind = np.arange(int(N * T / 3), int(N * (T * 2 / 3 + 1)))
            xtest = c[ind, :]
            ytest = r1[ind]
            testper = per[ind]

            l1 = c.shape[0]
            l2 = len(r1)
            l3 = l2 - np.sum(np.isnan(r1))

            ind = np.arange(int(N * T * 2 / 3), min(l1, l2, l3))
            xoos = c[ind, :]
            yoos = r1[ind]

            # Monthly Demean
            ytrain_demean = ytrain - np.mean(ytrain)
            ytest_demean = ytest - np.mean(ytest)
            mtrain = np.mean(ytrain)
            mtest = np.mean(ytest)

            # Start to train
            r2_oos = np.zeros(3)  # OOS R2
            r2_is = np.zeros(3)  # IS R2

            # Random Forest
            if nump == 50:
                lamv = np.arange(10, 101, 10)
            elif nump == 100:
                lamv = np.arange(10, 201, 20)
            ne = 100
            lamc = [2, 4, 8, 16, 32]
            r = np.zeros((len(lamv), len(lamc), 3))

            #for n1 in (range(len(lamv))):
             #   nf = lamv[n1]
              #  for n2 in range(len(lamc)):
               #     nn = lamc[n2]
                #    clf = RandomForestRegressor(
                 #       n_estimators=ne,
                  #      max_features=nf,
                   #     max_depth=nn
                    #)
                    #clf.fit(xtrain, ytrain)
                    #yhatbig1 = clf.predict(xtest)
                    #r[n1, n2, 0] = 1 - np.sum(np.power(yhatbig1 - ytest, 2)) / np.sum(
                     #   np.power(ytest - mtrain, 2))
                    #yhatbig1 = clf.predict(xoos)
                    #r[n1, n2, 1] = 1 - np.sum(np.power(yhatbig1 - yoos, 2)) / np.sum(
                     #   np.power(yoos - mtrain, 2))
                    #yhatbig1 = clf.predict(xtrain)
                    #r[n1, n2, 2] = 1 - np.sum(np.power(yhatbig1 - ytrain, 2)) / np.sum(
                     #   np.power(ytrain - mtrain, 2))
            ytest_mtrain_sq = np.power(ytest - mtrain, 2)
            yoos_mtrain_sq = np.power(yoos - mtrain, 2)
            ytrain_mtrain_sq = np.power(ytrain - mtrain, 2)
            for n1, nf in enumerate(lamv):
                for n2, nn in enumerate(lamc):
                    clf = RandomForestRegressor(
                        n_estimators=ne,
                        max_features=nf,
                        max_depth=nn
                    )
                    clf.fit(xtrain, ytrain)

                    # Predictions
                    yhat_test = clf.predict(xtest)
                    yhat_oos = clf.predict(xoos)
                    yhat_train = clf.predict(xtrain)

                    # Compute performance metrics
                    r[n1, n2, 0] = 1 - np.sum(np.power(yhat_test - ytest, 2)) / np.sum(ytest_mtrain_sq)
                    r[n1, n2, 1] = 1 - np.sum(np.power(yhat_oos - yoos, 2)) / np.sum(yoos_mtrain_sq)
                    r[n1, n2, 2] = 1 - np.sum(np.power(yhat_train - ytrain, 2)) / np.sum(ytrain_mtrain_sq)

            fw_2 = np.unravel_index(np.argmax(r[:, :, 0]), r[:, :, 0].shape)
            r2_oos[0] = r[fw_2[0], fw_2[1], 1]
            r2_is[0] = r[fw_2[0], fw_2[1], 2]
            print(f"RF R2 : {r2_oos[0]:.3f}")
            
            
            #GBRT
            
            lamv = np.arange(-1, 0.1, 0.2)
            r = np.zeros((len(lamv), 50, 3))

            for n1 in range(len(lamv)):
                lr = 10 ** lamv[n1]
                alpha = 2
                ne = 50
                clf = GradientBoostingRegressor(
                    n_estimators=ne,
                    learning_rate=lr,
                    loss='ls',
                    max_depth=2
                )

                clf.fit(xtrain, ytrain)
                e = clf.staged_predict(xtest)
                for i, pred in enumerate(e):
                    r[n1, i, 0] = np.mean((pred - ytest) ** 2)

                e = clf.staged_predict(xoos)
                for i, pred in enumerate(e):
                    r[n1, i, 1] = np.mean((pred - yoos) ** 2)

                e = clf.staged_predict(xtrain)
                for i, pred in enumerate(e):
                    r[n1, i, 2] = np.mean((pred - ytrain) ** 2)

            fw_2 = np.unravel_index(np.argmin(r[:, :, 0]), r[:, :, 0].shape)
            err1 = np.mean((ytrain - mtrain) ** 2)
            err2 = np.mean((yoos - mtrain) ** 2)
            r2_oos[1] = 1 - r[fw_2[0], fw_2[1], 1] / err2
            r2_is[1] = 1 - r[fw_2[0], fw_2[1], 2] / err1
            print(f"GBRT R2 : {r2_oos[1]:.3f}")

            # Save r2_oos and r2_is to files
            pathr = f"{title}/roos_{mo}_{M}.csv"
            np.savetxt(pathr, r2_oos.reshape(1, -1), delimiter=",")
            pathr = f"{title}/ris_{mo}_{M}.csv"
            np.savetxt(pathr, r2_is.reshape(1, -1), delimiter=",")

In [ ]:
MC = 1  # setup MC number
datanum = 50  # Or datanum = 100; separately run two cases
path = './Simu'  # set your own folder path
dirstock = os.path.join(path, f'SimuData_p{datanum}/')
hh = [1]
mo = 1
# for hh in [1, 3, 6, 12]:  # correspond to monthly quarterly halfyear and annually returns
title = os.path.join(path, f'Simu_p{datanum}/Tree{hh}')

if not os.path.isdir(title) and MC == 1:
    os.makedirs(title)

titleB = os.path.join(title, 'B')
if not os.path.isdir(titleB) and MC == 1:
    os.makedirs(titleB)

if datanum == 50:
    nump = 50
if datanum == 100:
    nump = 100
N = 200  # Number of CS tickers
m = nump * 2  # Number of Characteristics
T = 180  # Number of Time Periods

per = np.tile(np.arange(1, N + 1), T)
time = np.repeat(np.arange(1, T + 1), N)
stdv = 0.05
theta_w = 0.005

# Read Files
path1 = f"{dirstock}c{M}.csv"
path2 = f"{dirstock}r{mo}_{M}.csv"
c = np.genfromtxt(path1, delimiter=',')
r1 = np.genfromtxt(path2, delimiter=',')

# Add Some Elements
daylen = np.tile(N, T // 3)
daylen_test = daylen
ind = np.arange(0, int(N * T / 3))
xtrain = c[ind, :]
ytrain = r1[ind]
trainper = per[ind]
ind = np.arange(int(N * T / 3), int(N * (T * 2 / 3 + 1)))
xtest = c[ind, :]
ytest = r1[ind]
testper = per[ind]

l1 = c.shape[0]
l2 = len(r1)
l3 = l2 - np.sum(np.isnan(r1))

ind = np.arange(int(N * T * 2 / 3), min(l1, l2, l3))
xoos = c[ind, :]
yoos = r1[ind]

# Monthly Demean
ytrain_demean = ytrain - np.mean(ytrain)
ytest_demean = ytest - np.mean(ytest)
mtrain = np.mean(ytrain)
mtest = np.mean(ytest)

# Start to train
r2_oos = np.zeros(3)  # OOS R2
r2_is = np.zeros(3)  # IS R2

# Random Forest
if nump == 50:
    lamv = np.arange(10, 101, 10)
elif nump == 100:
    lamv = np.arange(10, 201, 20)
ne = 100
lamc = [2, 4, 8, 16, 32]
r = np.zeros((len(lamv), len(lamc), 3))

for n1 in tqdm(range(len(lamv))):
    nf = lamv[n1]
    for n2 in range(len(lamc)):
        nn = lamc[n2]
        clf = RandomForestRegressor(
            n_estimators=ne,
            max_features=nf,
            max_depth=nn
        )
        clf.fit(xtrain, ytrain)
        yhatbig1 = clf.predict(xtest)
        r[n1, n2, 0] = 1 - np.sum(np.power(yhatbig1 - ytest, 2)) / np.sum(
        np.power(ytest - mtrain, 2))
        yhatbig1 = clf.predict(xoos)
        r[n1, n2, 1] = 1 - np.sum(np.power(yhatbig1 - yoos, 2)) / np.sum(
        np.power(yoos - mtrain, 2))
        yhatbig1 = clf.predict(xtrain)
        r[n1, n2, 2] = 1 - np.sum(np.power(yhatbig1 - ytrain, 2)) / np.sum(
        np.power(ytrain - mtrain, 2))

fw_2 = np.unravel_index(np.argmax(r[:, :, 0]), r[:, :, 0].shape)
r2_oos[0] = r[fw_2[0], fw_2[1], 1]
r2_is[0] = r[fw_2[0], fw_2[1], 2]
print(f"RF R2 : {r2_oos[0]:.3f}")

# Save r2_oos and r2_is to files
pathr = f"{title}/roos_{mo}_{M}.csv"
np.savetxt(pathr, r2_oos.reshape(1, -1), delimiter=",")
pathr = f"{title}/ris_{mo}_{M}.csv"
np.savetxt(pathr, r2_is.reshape(1, -1), delimiter=",")

In [ ]:
#Gradient Boosting 
mo = 1

lamv = np.arange(-1, 0.1, 0.2)
r = np.zeros((len(lamv), 50, 3))

for n1 in range(len(lamv)):
    lr = 10 ** lamv[n1]
    alpha = 2
    ne = 50
    clf = GradientBoostingRegressor(
        n_estimators=ne,
        learning_rate=lr,
        loss='huber',
        max_depth=2
    )

    clf.fit(xtrain, ytrain)
    e = clf.staged_predict(xtest)
    for i, pred in enumerate(e):
        r[n1, i, 0] = np.mean((pred - ytest) ** 2)

    e = clf.staged_predict(xoos)
    for i, pred in enumerate(e):
        r[n1, i, 1] = np.mean((pred - yoos) ** 2)

    e = clf.staged_predict(xtrain)
    for i, pred in enumerate(e):
        r[n1, i, 2] = np.mean((pred - ytrain) ** 2)

fw_2 = np.unravel_index(np.argmin(r[:, :, 0]), r[:, :, 0].shape)
err1 = np.mean((ytrain - mtrain) ** 2)
err2 = np.mean((yoos - mtrain) ** 2)
r2_oos[1] = 1 - r[fw_2[0], fw_2[1], 1] / err2
r2_is[1] = 1 - r[fw_2[0], fw_2[1], 2] / err1
print(f"GBRT R2 : {r2_oos[1]:.3f}")

# Save r2_oos and r2_is to files
pathr = f"{title}/roos_{mo}_{M}.csv"
np.savetxt(pathr, r2_oos.reshape(1, -1), delimiter=",")
pathr = f"{title}/ris_{mo}_{M}.csv"
np.savetxt(pathr, r2_is.reshape(1, -1), delimiter=",")

# Additional Manipulations to get top-1000 and bottom-1000 of simulated stocks

In [ ]:
x_c.shape, x_b.shape

In [ ]:
updated = pd.concat([x_c, x_b], ignore_index=True)
#assume that the 4th column is the Market Value of the stock. Proceeding to arrange it in accordance to the column values
updated = updated.sort_values(by=updated.columns[3], ascending=False)
rtop_1 = updated.iloc[:,:1]
ctop1 = updated.iloc[:,1:]

rtop_1 = rtop_1.dropna()
ctop1 = ctop1.dropna()



In [ ]:
rtop_1.to_csv('r1_1.csv', index=False, header=False)
ctop1.to_csv('c1.csv', index=False, header=False)

Same simulations for top 1000

In [ ]:
MC = 1  # setup MC number
datanum = 50  # Or datanum = 100; separately run two cases
path = './Simu'  # set your own folder path    
dirstock = f"{path}/SimuData_d{datanum}/"

hh = [1]
# hh = [1, 3, 6, 12]  # correspond to monthly, quarterly, half-year, and annually returns
title = f"{path}/Simu_p{datanum}/Reg{hh[0]}"
if not os.path.exists(title) and MC == 1:
    os.makedirs(title)
titleB = f"{title}/B"
if not os.path.exists(titleB) and MC == 1:
    os.makedirs(titleB)
if datanum == 50:
    nump = 50
elif datanum == 100:
    nump = 100
mu = 0.2 * np.sqrt(hh[0])
tol = 10**(-10)

for M in range(1, 2):  # Assuming range for M
    for mo in range(1, 2):  # Assuming range for mo
        
        print('### MCMC : {}, Model : {} ###'.format(M, mo))
        
        N = 200   # Number of CS tickers
        m = nump * 2   # Number of Characteristics
        T = 4  # Number of Time Periods
        
        per = np.tile(np.arange(1, N+1), T)
        time = np.repeat(np.arange(1, T+1), N)
        stdv = 0.05
        theta_w = 0.005
        
        # Read Files
        path1 = dirstock + 'c' + str(M) + '.csv'
        path2 = dirstock + 'r' + str(mo) + '_' + str(M) + '.csv'
        c = np.genfromtxt(path1, delimiter=',')
        r1 = np.genfromtxt(path2, delimiter=',')
        
        # Add Some Elements
        daylen = np.repeat(N, T//3)
        daylen_test = daylen.copy()
        ind = np.arange(0, N*T//3)
        xtrain = c[ind]
        ytrain = r1[ind]
        trainper = per[ind]
        start_idx = math.floor(N * T / 3) + 1
        end_idx = math.floor(N * (T * 2 / 3 - hh[0] + 1))
        ind = list(range(start_idx, end_idx))
        xtest = c[ind]
        ytest = r1[ind]
        testper = per[ind]
        
        l1 = c.shape[0]
        l2 = len(r1)
        l3 = l2 - np.isnan(r1).sum()
        
        ind = np.arange(N*(2*T//3), min([l1, l2, l3]))
        xoos = c[ind]
        yoos = r1[ind]
        
        # Monthly Demean
        ytrain_demean = ytrain - np.mean(ytrain)
        ytest_demean = ytest - np.mean(ytest)
        mtrain = np.mean(ytrain)
        mtest = np.mean(ytest)
        
        # Calculate Sufficient Stats
        sd = np.zeros(xtrain.shape[1])
        for i in range(xtrain.shape[1]):
            s = np.std(xtrain[:, i])
            if s > 0:
                xtrain[:, i] /= s
                xtest[:, i] /= s
                xoos[:, i] /= s
                sd[i] = s
        
        XX = np.dot(xtrain.T, xtrain)
        U, S, V = np.linalg.svd(XX)
        L = S[0]
        Y = ytrain_demean
        XY = np.dot(xtrain.T, Y)
        
         
        #OLS
        
        # Initialize arrays
        r2_oos = np.zeros(13)
        r2_is = np.zeros(13)
        modeln = 0
        groups = 0
        nc = 0

        # OLS
        modeln += 1
        clf = LinearRegression(fit_intercept=False)
        clf.fit(xtrain, ytrain_demean)
        yhatbig1 = clf.predict(xoos) + mtrain
        r2_oos[modeln - 1] = 1 - np.sum(np.power(yhatbig1 - yoos, 2)) / np.sum(np.power(yoos - mtrain, 2))
        yhatbig1 = clf.predict(xtrain) + mtrain
        r2_is[modeln - 1] = 1 - np.sum(np.power(yhatbig1 - ytrain, 2)) / np.sum(np.power(ytrain - mtrain, 2))
        b = clf.coef_
        pathb = f"{title}/B/b{mo}_{M}_{modeln}.csv"
        pd.DataFrame(b).to_csv(pathb, index=False, header=False)
        print(f"Simple OLS R2: {r2_oos[modeln - 1]:.3f}")

        # OLS+H
        modeln += 1
        func = soft_threshodl
        b = proximalH(groups, nc, xtest, mtrain, ytest, b, xtrain, ytrain_demean, mu, tol, L, 0, func)
        yhatbig1 = np.dot(xoos, b) + mtrain
        r2_oos[modeln - 1] = 1 - np.sum(np.power(yhatbig1[:1000] - yoos[:1000], 2)) / np.sum(np.power(yoos[:1000] - mtrain, 2))
        yhatbig1 = np.dot(xtrain, b) + mtrain
        r2_is[modeln - 1] = 1 - np.sum(np.power(yhatbig1[:1000] - ytrain[:1000], 2)) / np.sum(np.power(ytrain[:1000] - mtrain, 2))
        pathb = f"{title}/B/b{mo}_{M}_{modeln}.csv"
        pd.DataFrame(b).to_csv(pathb, index=False, header=False)
        print(f"Simple OLS+H R2: {r2_oos[modeln - 1]:.3f}")

        # PCR
        modeln += 1
        ne = 30
        X = np.dot(xtrain.T, xtrain)
        pca_vec = V.T
        p1 = pca_vec[:, :ne]
        Z = np.dot(xtrain, p1)
        r = np.zeros((3, ne))
        B = np.zeros((xtrain.shape[1], ne))
        Y = ytrain_demean

        for j in range(ne - 1):
            xx = Z[:, :j + 1]
            b = np.dot(np.linalg.inv(np.dot(xx.T, xx)), np.dot(xx.T, Y))
            b = np.dot(p1[:, :j + 1], b)

            yhatbig1 = np.dot(xtest, b) + mtrain
            r[0, j] = 1 - np.sum(np.power(yhatbig1[:1000] - ytest[:1000], 2)) / np.sum(np.power(ytest[:1000] - mtrain, 2))
            yhatbig1 = np.dot(xoos, b) + mtrain
            r[1, j] = 1 - np.sum(np.power(yhatbig1[:1000] - yoos[:1000], 2)) / np.sum(np.power(yoos[:1000] - mtrain, 2))
            yhatbig1 = np.dot(xtrain, b) + mtrain
            r[2, j] = 1 - np.sum(np.power(yhatbig1[:1000] - ytrain[:1000], 2)) / np.sum(np.power(ytrain[:1000] - mtrain, 2))
            B[:, j] = b

        b = np.zeros(xtest.shape[1])
        j = ne - 1
        yhatbig1 = np.dot(xtest, b) + mtrain
        r[0, j] = 1 - np.sum(np.power(yhatbig1[:1000] - ytest[:1000], 2)) / np.sum(np.power(ytest[:1000] - mtrain, 2))
        yhatbig1 = np.dot(xoos, b) + mtrain
        r[1, j] = 1 - np.sum(np.power(yhatbig1[:1000] - yoos[:1000], 2)) / np.sum(np.power(yoos[:1000] - mtrain, 2))
        yhatbig1 = np.dot(xtrain, b) + mtrain
        r[2, j] = 1 - np.sum(np.power(yhatbig1[:1000] - ytrain[:1000], 2)) / np.sum(np.power(ytrain[:1000] - mtrain, 2))
        B[:, j] = b

        r2_oos[modeln - 1] = r[1, fw1(r[0, :].tolist())]
        r2_is[modeln - 1] = r[2, fw1(r[0, :].tolist())]
        b = B[:, fw1(r[0, :].tolist())]
        pathb = f"{title}/B/b{mo}_{M}_{modeln}.csv"
        pd.DataFrame(b).to_csv(pathb, index=False, header=False)
        print(f"PCR R2: {r2_oos[modeln - 1]:.3f}")

        # PLS
        modeln += 1
        B = pls(xtrain, ytrain_demean, 30)
        ne = 30
        r = np.zeros((3, ne))
        Y = ytrain_demean

        for j in range(ne):
            b = B[:, j]
            yhatbig1 = np.dot(xtest, b) + mtrain
            r[0, j] = 1 - np.sum(np.power(yhatbig1[:1000] - ytest[:1000], 2)) / np.sum(np.power(ytest[:1000] - mtrain, 2))
            yhatbig1 = np.dot(xoos, b) + mtrain
            r[1, j] = 1 - np.sum(np.power(yhatbig1[:1000] - yoos[:1000], 2)) / np.sum(np.power(yoos[:1000] - mtrain, 2))
            yhatbig1 = np.dot(xtrain, b) + mtrain
            r[2, j] = 1 - np.sum(np.power(yhatbig1[:1000] - ytrain[:1000], 2)) / np.sum(np.power(ytrain[:1000] - mtrain, 2))

        r2_oos[modeln - 1] = r[1, fw1(r[0, :].tolist())]
        r2_is[modeln - 1] = r[2, fw1(r[0, :].tolist())]
        b = B[:, fw1(r[0, :].tolist())]
        pathb = f"{title}/B/b{mo}_{M}_{modeln}.csv"
        pd.DataFrame(b).to_csv(pathb, index=False, header=False)
        print(f"PLS R2: {r2_oos[modeln - 1]:.3f}")
            
        # Elastic Net
        modeln += 1
        lamv = np.arange(-2, 4.1, 0.1)
        alpha = 0.5
        r = np.zeros((3, len(lamv)))

        for j in range(len(lamv)):
            l2 = 10 ** lamv[j]
            func = soft_threshode
            b = proximal(groups, nc, XX, XY, tol, L, l2, func)
            yhatbig1 = np.dot(xtest, b) + mtrain
            r[0, j] = 1 - np.sum(np.power(yhatbig1[:1000] - ytest[:1000], 2)) / np.sum(np.power(ytest[:1000] - mtrain, 2))
            yhatbig1 = np.dot(xoos, b) + mtrain
            r[1, j] = 1 - np.sum(np.power(yhatbig1[:1000] - yoos[:1000], 2)) / np.sum(np.power(yoos[:1000] - mtrain, 2))
            yhatbig1 = np.dot(xtrain, b) + mtrain
            r[2, j] = 1 - np.sum(np.power(yhatbig1[:1000] - ytrain[:1000], 2)) / np.sum(np.power(ytrain[:1000] - mtrain, 2))

        r2_oos[modeln - 1] = r[1, fw1(r[0, :].tolist())]
        r2_is[modeln - 1] = r[2, fw1(r[0, :].tolist())]
        l2 = 10 ** lamv[int(fw1(r[0]))]
        func = soft_threshode
        b = proximal(groups, nc, XX, XY, tol, L, l2, func)
        pathb = f"{title}/B/b{mo}_{M}_{modeln}.csv"
        np.savetxt(pathb, b, delimiter=',')
        print('Enet R2 :', np.round(r2_oos[modeln-1], 3))

        modeln += 1
        func = soft_threshode
        b = proximalH(groups, nc, xtest, mtrain, ytest, b, xtrain, ytrain_demean, mu, tol, L, l2, func)
        yhatbig1 = np.dot(xoos, b) + mtrain
        r2_oos[modeln-1] = 1 - np.sum(np.power(yhatbig1[:1000] - yoos[:1000], 2)) / np.sum(np.power(yoos[:1000] - mtrain, 2))
        yhatbig1 = np.dot(xtrain, b) + mtrain
        r2_is[modeln-1] = 1 - np.sum(np.power(yhatbig1[:1000] - ytrain[:1000], 2)) / np.sum(np.power(ytrain[:1000] - mtrain, 2))
        pathb = f"{title}/B/b{mo}_{M}_{modeln}.csv"
        np.savetxt(pathb, b, delimiter=',')
        print('Enet+H R2 :', np.round(r2_oos[modeln-1], 3))
        
        # Group Lasso
        kn = 4
        th = np.zeros((kn, xtrain.shape[1]))
        th[1, :] = 0

        for i in range(xtrain.shape[1]):
            th[:, i] = np.quantile(xtrain[:, i], np.arange(kn) / kn)

        xtrain = cut_knots_degree2(xtrain, kn, th)
        xtest = cut_knots_degree2(xtest, kn, th)
        xoos = cut_knots_degree2(xoos, kn, th)

        for i in range(xtrain.shape[1]):
            s = np.std(xtrain[:, i])
            if s > 0:
                xtrain[:, i] = xtrain[:, i] / s
                xtest[:, i] = xtest[:, i] / s
                xoos[:, i] = xoos[:, i] / s

        Y = ytrain_demean
        XX = xtrain.T @ xtrain
        U, S, V = np.linalg.svd(XX)
        L = S[0]
        XY = xtrain.T @ Y

        modeln += 1
        lamv = np.arange(0.5, 3.1, 0.1)
        nc = XX.shape[1] // (kn + 1)
        groups = np.repeat(np.arange(1, nc + 1), kn + 1)
        r = np.zeros((3, len(lamv)))

        for j, lam in enumerate(lamv):
            l2 = 10 ** lam
            func = soft_threshodg
            b = proximal(groups, nc, XX, XY, tol, L, l2, func)
            yhatbig1 = xtest @ b + mtrain
            r[0, j] = 1 - np.sum((yhatbig1 - ytest) ** 2) / np.sum((ytest - mtrain) ** 2)
            yhatbig1 = xoos @ b + mtrain
            r[1, j] = 1 - np.sum((yhatbig1 - yoos) ** 2) / np.sum((yoos - mtrain) ** 2)
            yhatbig1 = xtrain @ b + mtrain
            r[2, j] = 1 - np.sum((yhatbig1 - ytrain) ** 2) / np.sum((ytrain - mtrain) ** 2)

        r2_oos[modeln] = r[1, np.int16(fw1(r[0, :]))]
        r2_is[modeln] = r[2, np.int16(fw1(r[0, :]))]
        l2 = 10 ** lamv[np.int16(fw1(r[0, :]))]

        func = soft_threshodg
        b = proximal(groups, nc, XX, XY, tol, L, l2, func)
        pathb = f"{title}/B/b"
        pathb = f"{pathb}_{mo}_{M}_{modeln}.csv"
        np.savetxt(pathb, b, delimiter=",")

        print("Group Lasso R2:", np.round(r2_oos[modeln], 3))

        modeln += 1
        func = soft_threshodg
        b = proximalH(groups, nc, xtest, mtrain, ytest, b, xtrain, ytrain_demean, mu, tol, L, l2, func)
        yhatbig1 = xoos @ b + mtrain
        r2_oos[modeln] = 1 - np.sum((yhatbig1[:1000] - yoos[:1000]) ** 2) / np.sum((yoos[:1000] - mtrain) ** 2)
        yhatbig1 = xtrain @ b + mtrain
        r2_is[modeln] = 1 - np.sum((yhatbig1[:1000] - ytrain[:1000]) ** 2) / np.sum((ytrain[:1000] - mtrain) ** 2)
        pathb = f"{title}/B/b"
        pathb = f"{pathb}_{mo}_{M}_{modeln}.csv"
        np.savetxt(pathb, b, delimiter=",")

        print("Group Lasso+H R2:", np.round(r2_oos[modeln], 3))

        pathr = f"{title}/roos"
        pathr = f"{pathr}_{mo}_{M}.csv"
        np.savetxt(pathr, r2_oos, delimiter=",")

        pathr = f"{title}/ris"
        pathr = f"{pathr}_{mo}_{M}.csv"
        np.savetxt(pathr, r2_is, delimiter=",")

And for the bottom 1000

In [ ]:
MC = 1  # setup MC number
datanum = 50  # Or datanum = 100; separately run two cases
path = './Simu'  # set your own folder path    
dirstock = f"{path}/SimuData_d{datanum}/"

hh = [1]
# hh = [1, 3, 6, 12]  # correspond to monthly, quarterly, half-year, and annually returns
title = f"{path}/Simu_p{datanum}/Reg{hh[0]}"
if not os.path.exists(title) and MC == 1:
    os.makedirs(title)
titleB = f"{title}/B"
if not os.path.exists(titleB) and MC == 1:
    os.makedirs(titleB)
if datanum == 50:
    nump = 50
elif datanum == 100:
    nump = 100
mu = 0.2 * np.sqrt(hh[0])
tol = 10**(-10)

for M in range(1, 2):  # Assuming range for M
    for mo in range(1, 2):  # Assuming range for mo
        
        print('### MCMC : {}, Model : {} ###'.format(M, mo))
        
        N = 200   # Number of CS tickers
        m = nump * 2   # Number of Characteristics
        T = 4  # Number of Time Periods
        
        per = np.tile(np.arange(1, N+1), T)
        time = np.repeat(np.arange(1, T+1), N)
        stdv = 0.05
        theta_w = 0.005
        
        # Read Files
        path1 = dirstock + 'c' + str(M) + '.csv'
        path2 = dirstock + 'r' + str(mo) + '_' + str(M) + '.csv'
        c = np.genfromtxt(path1, delimiter=',')
        r1 = np.genfromtxt(path2, delimiter=',')
        
        # Add Some Elements
        daylen = np.repeat(N, T//3)
        daylen_test = daylen.copy()
        ind = np.arange(0, N*T//3)
        xtrain = c[ind]
        ytrain = r1[ind]
        trainper = per[ind]
        start_idx = math.floor(N * T / 3) + 1
        end_idx = math.floor(N * (T * 2 / 3 - hh[0] + 1))
        ind = list(range(start_idx, end_idx))
        xtest = c[ind]
        ytest = r1[ind]
        testper = per[ind]
        
        l1 = c.shape[0]
        l2 = len(r1)
        l3 = l2 - np.isnan(r1).sum()
        
        ind = np.arange(N*(2*T//3), min([l1, l2, l3]))
        xoos = c[ind]
        yoos = r1[ind]
        
        # Monthly Demean
        ytrain_demean = ytrain - np.mean(ytrain)
        ytest_demean = ytest - np.mean(ytest)
        mtrain = np.mean(ytrain)
        mtest = np.mean(ytest)
        
        # Calculate Sufficient Stats
        sd = np.zeros(xtrain.shape[1])
        for i in range(xtrain.shape[1]):
            s = np.std(xtrain[:, i])
            if s > 0:
                xtrain[:, i] /= s
                xtest[:, i] /= s
                xoos[:, i] /= s
                sd[i] = s
        
        XX = np.dot(xtrain.T, xtrain)
        U, S, V = np.linalg.svd(XX)
        L = S[0]
        Y = ytrain_demean
        XY = np.dot(xtrain.T, Y)
        
         
        #OLS
        
        # Initialize arrays
        r2_oos = np.zeros(13)
        r2_is = np.zeros(13)
        modeln = 0
        groups = 0
        nc = 0

        # OLS
        modeln += 1
        clf = LinearRegression(fit_intercept=False)
        clf.fit(xtrain, ytrain_demean)
        yhatbig1 = clf.predict(xoos) + mtrain
        r2_oos[modeln - 1] = 1 - np.sum(np.power(yhatbig1 - yoos, 2)) / np.sum(np.power(yoos - mtrain, 2))
        yhatbig1 = clf.predict(xtrain) + mtrain
        r2_is[modeln - 1] = 1 - np.sum(np.power(yhatbig1 - ytrain, 2)) / np.sum(np.power(ytrain - mtrain, 2))
        b = clf.coef_
        pathb = f"{title}/B/b{mo}_{M}_{modeln}.csv"
        pd.DataFrame(b).to_csv(pathb, index=False, header=False)
        print(f"Simple OLS R2: {r2_oos[modeln - 1]:.3f}")

        # OLS+H
        modeln += 1
        func = soft_threshodl
        b = proximalH(groups, nc, xtest, mtrain, ytest, b, xtrain, ytrain_demean, mu, tol, L, 0, func)
        yhatbig1 = np.dot(xoos, b) + mtrain
        r2_oos[modeln - 1] = 1 - np.sum(np.power(yhatbig1[-1000:] - yoos[-1000:], 2)) / np.sum(np.power(yoos[-1000:]- mtrain, 2))
        yhatbig1 = np.dot(xtrain, b) + mtrain
        r2_is[modeln - 1] = 1 - np.sum(np.power(yhatbig1[-1000:] - ytrain[-1000:], 2)) / np.sum(np.power(ytrain[-1000:]- mtrain, 2))
        pathb = f"{title}/B/b{mo}_{M}_{modeln}.csv"
        pd.DataFrame(b).to_csv(pathb, index=False, header=False)
        print(f"Simple OLS+H R2: {r2_oos[modeln - 1]:.3f}")

        # PCR
        modeln += 1
        ne = 30
        X = np.dot(xtrain.T, xtrain)
        pca_vec = V.T
        p1 = pca_vec[:, :ne]
        Z = np.dot(xtrain, p1)
        r = np.zeros((3, ne))
        B = np.zeros((xtrain.shape[1], ne))
        Y = ytrain_demean

        for j in range(ne - 1):
            xx = Z[:, :j + 1]
            b = np.dot(np.linalg.inv(np.dot(xx.T, xx)), np.dot(xx.T, Y))
            b = np.dot(p1[:, :j + 1], b)

            yhatbig1 = np.dot(xtest, b) + mtrain
            r[0, j] = 1 - np.sum(np.power(yhatbig1[-1000:] - ytest[-1000:], 2)) / np.sum(np.power(ytest[-1000:] - mtrain, 2))
            yhatbig1 = np.dot(xoos, b) + mtrain
            r[1, j] = 1 - np.sum(np.power(yhatbig1[-1000:] - yoos[-1000:], 2)) / np.sum(np.power(yoos[-1000:] - mtrain, 2))
            yhatbig1 = np.dot(xtrain, b) + mtrain
            r[2, j] = 1 - np.sum(np.power(yhatbig1[-1000:] - ytrain[-1000:], 2)) / np.sum(np.power(ytrain[-1000:] - mtrain, 2))
            B[:, j] = b

        b = np.zeros(xtest.shape[1])
        j = ne - 1
        yhatbig1 = np.dot(xtest, b) + mtrain
        r[0, j] = 1 - np.sum(np.power(yhatbig1[-1000:] - ytest[-1000:], 2)) / np.sum(np.power(ytest[-1000:] - mtrain, 2))
        yhatbig1 = np.dot(xoos, b) + mtrain
        r[1, j] = 1 - np.sum(np.power(yhatbig1[-1000:] - yoos[-1000:], 2)) / np.sum(np.power(yoos[-1000:] - mtrain, 2))
        yhatbig1 = np.dot(xtrain, b) + mtrain
        r[2, j] = 1 - np.sum(np.power(yhatbig1[-1000:] - ytrain[-1000:], 2)) / np.sum(np.power(ytrain[-1000:] - mtrain, 2))
        B[:, j] = b

        r2_oos[modeln - 1] = r[1, fw1(r[0, :].tolist())]
        r2_is[modeln - 1] = r[2, fw1(r[0, :].tolist())]
        b = B[:, fw1(r[0, :].tolist())]
        pathb = f"{title}/B/b{mo}_{M}_{modeln}.csv"
        pd.DataFrame(b).to_csv(pathb, index=False, header=False)
        print(f"PCR R2: {r2_oos[modeln - 1]:.3f}")

        # PLS
        modeln += 1
        B = pls(xtrain, ytrain_demean, 30)
        ne = 30
        r = np.zeros((3, ne))
        Y = ytrain_demean

        for j in range(ne):
            b = B[:, j]
            yhatbig1 = np.dot(xtest, b) + mtrain
            r[0, j] = 1 - np.sum(np.power(yhatbig1[-1000:] - ytest[-1000:], 2)) / np.sum(np.power(ytest[-1000:] - mtrain, 2))
            yhatbig1 = np.dot(xoos, b) + mtrain
            r[1, j] = 1 - np.sum(np.power(yhatbig1[-1000:] - yoos[-1000:], 2)) / np.sum(np.power(yoos[-1000:] - mtrain, 2))
            yhatbig1 = np.dot(xtrain, b) + mtrain
            r[2, j] = 1 - np.sum(np.power(yhatbig1[-1000:] - ytrain[-1000:], 2)) / np.sum(np.power(ytrain[-1000:] - mtrain, 2))

        r2_oos[modeln - 1] = r[1, fw1(r[0, :].tolist())]
        r2_is[modeln - 1] = r[2, fw1(r[0, :].tolist())]
        b = B[:, fw1(r[0, :].tolist())]
        pathb = f"{title}/B/b{mo}_{M}_{modeln}.csv"
        pd.DataFrame(b).to_csv(pathb, index=False, header=False)
        print(f"PLS R2: {r2_oos[modeln - 1]:.3f}")
            
        # Elastic Net
        modeln += 1
        lamv = np.arange(-2, 4.1, 0.1)
        alpha = 0.5
        r = np.zeros((3, len(lamv)))

        for j in range(len(lamv)):
            l2 = 10 ** lamv[j]
            func = soft_threshode
            b = proximal(groups, nc, XX, XY, tol, L, l2, func)
            yhatbig1 = np.dot(xtest, b) + mtrain
            r[0, j] = 1 - np.sum(np.power(yhatbig1[-1000:] - ytest[-1000:], 2)) / np.sum(np.power(ytest[-1000:] - mtrain, 2))
            yhatbig1 = np.dot(xoos, b) + mtrain
            r[1, j] = 1 - np.sum(np.power(yhatbig1[-1000:] - yoos[-1000:], 2)) / np.sum(np.power(yoos[-1000:] - mtrain, 2))
            yhatbig1 = np.dot(xtrain, b) + mtrain
            r[2, j] = 1 - np.sum(np.power(yhatbig1[-1000:] - ytrain[-1000:], 2)) / np.sum(np.power(ytrain[-1000:] - mtrain, 2))

        r2_oos[modeln - 1] = r[1, fw1(r[0, :].tolist())]
        r2_is[modeln - 1] = r[2, fw1(r[0, :].tolist())]
        l2 = 10 ** lamv[int(fw1(r[0]))]
        func = soft_threshode
        b = proximal(groups, nc, XX, XY, tol, L, l2, func)
        pathb = f"{title}/B/b{mo}_{M}_{modeln}.csv"
        np.savetxt(pathb, b, delimiter=',')
        print('Enet R2 :', np.round(r2_oos[modeln-1], 3))

        modeln += 1
        func = soft_threshode
        b = proximalH(groups, nc, xtest, mtrain, ytest, b, xtrain, ytrain_demean, mu, tol, L, l2, func)
        yhatbig1 = np.dot(xoos, b) + mtrain
        r2_oos[modeln-1] = 1 - np.sum(np.power(yhatbig1[-1000:] - yoos[-1000:], 2)) / np.sum(np.power(yoos[-1000:] - mtrain, 2))
        yhatbig1 = np.dot(xtrain, b) + mtrain
        r2_is[modeln-1] = 1 - np.sum(np.power(yhatbig1[-1000:] - ytrain[-1000:], 2)) / np.sum(np.power(ytrain[-1000:] - mtrain, 2))
        pathb = f"{title}/B/b{mo}_{M}_{modeln}.csv"
        np.savetxt(pathb, b, delimiter=',')
        print('Enet+H R2 :', np.round(r2_oos[modeln-1], 3))
        
        # Group Lasso
        kn = 4
        th = np.zeros((kn, xtrain.shape[1]))
        th[1, :] = 0

        for i in range(xtrain.shape[1]):
            th[:, i] = np.quantile(xtrain[:, i], np.arange(kn) / kn)

        xtrain = cut_knots_degree2(xtrain, kn, th)
        xtest = cut_knots_degree2(xtest, kn, th)
        xoos = cut_knots_degree2(xoos, kn, th)

        for i in range(xtrain.shape[1]):
            s = np.std(xtrain[:, i])
            if s > 0:
                xtrain[:, i] = xtrain[:, i] / s
                xtest[:, i] = xtest[:, i] / s
                xoos[:, i] = xoos[:, i] / s

        Y = ytrain_demean
        XX = xtrain.T @ xtrain
        U, S, V = np.linalg.svd(XX)
        L = S[0]
        XY = xtrain.T @ Y

        modeln += 1
        lamv = np.arange(0.5, 3.1, 0.1)
        nc = XX.shape[1] // (kn + 1)
        groups = np.repeat(np.arange(1, nc + 1), kn + 1)
        r = np.zeros((3, len(lamv)))

        for j, lam in enumerate(lamv):
            l2 = 10 ** lam
            func = soft_threshodg
            b = proximal(groups, nc, XX, XY, tol, L, l2, func)
            yhatbig1 = xtest @ b + mtrain
            r[0, j] = 1 - np.sum((yhatbig1 - ytest) ** 2) / np.sum((ytest - mtrain) ** 2)
            yhatbig1 = xoos @ b + mtrain
            r[1, j] = 1 - np.sum((yhatbig1 - yoos) ** 2) / np.sum((yoos - mtrain) ** 2)
            yhatbig1 = xtrain @ b + mtrain
            r[2, j] = 1 - np.sum((yhatbig1 - ytrain) ** 2) / np.sum((ytrain - mtrain) ** 2)

        r2_oos[modeln] = r[1, np.int16(fw1(r[0, :]))]
        r2_is[modeln] = r[2, np.int16(fw1(r[0, :]))]
        l2 = 10 ** lamv[np.int16(fw1(r[0, :]))]

        func = soft_threshodg
        b = proximal(groups, nc, XX, XY, tol, L, l2, func)
        pathb = f"{title}/B/b"
        pathb = f"{pathb}_{mo}_{M}_{modeln}.csv"
        np.savetxt(pathb, b, delimiter=",")

        print("Group Lasso R2:", np.round(r2_oos[modeln], 3))

        modeln += 1
        func = soft_threshodg
        b = proximalH(groups, nc, xtest, mtrain, ytest, b, xtrain, ytrain_demean, mu, tol, L, l2, func)
        yhatbig1 = xoos @ b + mtrain
        r2_oos[modeln] = 1 - np.sum((yhatbig1[-1000:] - yoos[-1000:]) ** 2) / np.sum((yoos[-1000:] - mtrain) ** 2)
        yhatbig1 = xtrain @ b + mtrain
        r2_is[modeln] = 1 - np.sum((yhatbig1[-1000:] - ytrain[-1000:]) ** 2) / np.sum((ytrain[-1000:] - mtrain) ** 2)
        pathb = f"{title}/B/b"
        pathb = f"{pathb}_{mo}_{M}_{modeln}.csv"
        np.savetxt(pathb, b, delimiter=",")

        print("Group Lasso+H R2:", np.round(r2_oos[modeln], 3))

        pathr = f"{title}/roos"
        pathr = f"{pathr}_{mo}_{M}.csv"
        np.savetxt(pathr, r2_oos, delimiter=",")

        pathr = f"{title}/ris"
        pathr = f"{pathr}_{mo}_{M}.csv"
        np.savetxt(pathr, r2_is, delimiter=",")

Now moving to boosing models

In [ ]:
MC = 1  # setup MC number
datanum = 50  # Or datanum = 100; separately run two cases
path = './Simu'  # set your own folder path
dirstock = os.path.join(path, f'SimuData_d{datanum}/')
hh = [1]
mo = 1
# for hh in [1, 3, 6, 12]:  # correspond to monthly quarterly halfyear and annually returns
title = os.path.join(path, f'Simu_p{datanum}/Tree{hh}')

if not os.path.isdir(title) and MC == 1:
    os.makedirs(title)

titleB = os.path.join(title, 'B')
if not os.path.isdir(titleB) and MC == 1:
    os.makedirs(titleB)

if datanum == 50:
    nump = 50
if datanum == 100:
    nump = 100
N = 200  # Number of CS tickers
m = nump * 2  # Number of Characteristics
T = 180  # Number of Time Periods

per = np.tile(np.arange(1, N + 1), T)
time = np.repeat(np.arange(1, T + 1), N)
stdv = 0.05
theta_w = 0.005

# Read Files
path1 = f"{dirstock}c{M}.csv"
path2 = f"{dirstock}r{mo}_{M}.csv"
c = np.genfromtxt(path1, delimiter=',')
r1 = np.genfromtxt(path2, delimiter=',')

# Add Some Elements
daylen = np.tile(N, T // 3)
daylen_test = daylen
ind = np.arange(0, int(N * T / 3))
xtrain = c[ind, :]
ytrain = r1[ind]
trainper = per[ind]
ind = np.arange(int(N * T / 3), int(N * (T * 2 / 3 + 1)))
xtest = c[ind, :]
ytest = r1[ind]
testper = per[ind]

l1 = c.shape[0]
l2 = len(r1)
l3 = l2 - np.sum(np.isnan(r1))

ind = np.arange(int(N * T * 2 / 3), min(l1, l2, l3))
xoos = c[ind, :]
yoos = r1[ind]

# Monthly Demean
ytrain_demean = ytrain - np.mean(ytrain)
ytest_demean = ytest - np.mean(ytest)
mtrain = np.mean(ytrain)
mtest = np.mean(ytest)

# Start to train
r2_oos = np.zeros(3)  # OOS R2
r2_is = np.zeros(3)
r3_oos = np.zeros(3) # IS R2

# Random Forest
if nump == 50:
    lamv = np.arange(10, 101, 10)
elif nump == 100:
    lamv = np.arange(10, 201, 20)
ne = 100
lamc = [2, 4, 8, 16, 32]
r = np.zeros((len(lamv), len(lamc), 3))
rrr = np.zeros((len(lamv), len(lamc), 3))

for n1 in tqdm(range(len(lamv))):
    nf = lamv[n1]
    for n2 in range(len(lamc)):
        nn = lamc[n2]
        clf = RandomForestRegressor(
            n_estimators=ne,
            max_features=nf,
            max_depth=nn
        )
        clf.fit(xtrain, ytrain)
        yhatbig1 = clf.predict(xtest)
        
        r[n1, n2, 0] = 1 - np.sum(np.power(yhatbig1[:1000] - ytest[:1000], 2)) / np.sum(
        np.power(ytest[:1000] - mtrain, 2))
        yhatbig1 = clf.predict(xoos)
        r[n1, n2, 1] = 1 - np.sum(np.power(yhatbig1[:1000] - yoos[:1000], 2)) / np.sum(
        np.power(yoos[:1000] - mtrain, 2))
        yhatbig1 = clf.predict(xtrain)
        r[n1, n2, 2] = 1 - np.sum(np.power(yhatbig1[:1000] - ytrain[:1000], 2)) / np.sum(
        np.power(ytrain[:1000] - mtrain, 2))
        
        rrr[n1, n2, 0] = 1 - np.sum(np.power(yhatbig1[-1000:] - ytest[-1000:], 2)) / np.sum(
        np.power(ytest[-1000:] - mtrain, 2))
        yhatbig1 = clf.predict(xoos)
        rrr[n1, n2, 1] = 1 - np.sum(np.power(yhatbig1[-1000:] - yoos[-1000:], 2)) / np.sum(
        np.power(yoos[-1000:] - mtrain, 2))
        yhatbig1 = clf.predict(xtrain)
        rrr[n1, n2, 2] = 1 - np.sum(np.power(yhatbig1[-1000:] - ytrain[-1000:], 2)) / np.sum(
        np.power(ytrain[-1000:] - mtrain, 2))

fw_2 = np.unravel_index(np.argmax(r[:, :, 0]), r[:, :, 0].shape)
fw_3 = np.unravel_index(np.argmax(rrr[:, :, 0]), rrr[:, :, 0].shape)

r2_oos[0] = r[fw_2[0], fw_2[1], 1]
r3_oos[0] = r[fw_3[0], fw_3[1], 1]
print(f"RF R2 for first 1000 : {r2_oos[0]:.3f}, RF R2 for last 1000 : {r3_oos[0]:.3f}")

In [ ]:
#Gradient Boosting 
mo = 1

lamv = np.arange(-1, 0.1, 0.2)
r = np.zeros((len(lamv), 50, 3))
rrr = np.zeros((len(lamv), 50, 3))

for n1 in range(len(lamv)):
    lr = 10 ** lamv[n1]
    alpha = 2
    ne = 50
    clf = GradientBoostingRegressor(
        n_estimators=ne,
        learning_rate=lr,
        loss='huber',
        max_depth=2
    )

    clf.fit(xtrain, ytrain)
    e = clf.staged_predict(xtest)
    for i, pred in enumerate(e):
        r[n1, i, 0] = np.mean((pred - ytest) ** 2)

    e = clf.staged_predict(xoos)
    for i, pred in enumerate(e):
        r[n1, i, 1] = np.mean((pred - yoos) ** 2)

    e = clf.staged_predict(xtrain)
    for i, pred in enumerate(e):
        r[n1, i, 2] = np.mean((pred[:1000] - ytrain[:1000]) ** 2)
        rrr[n1, i, 2] = np.mean((pred[-1000:] - ytrain[-1000:]) ** 2)
        

fw_2 = np.unravel_index(np.argmin(r[:, :, 0]), r[:, :, 0].shape)
fw_3 = np.unravel_index(np.argmin(rrr[:, :, 0]), rrr[:, :, 0].shape)
err2 = np.mean((yoos[:1000] - mtrain) ** 2)
err3 = np.mean((yoos[-1000:] - mtrain) ** 2)
r2_oos[1] = 1 - r[fw_2[0], fw_2[1], 1] / err2
r3_oos[1] = 1 - r[fw_3[0], fw_3[1], 1] / err3
print(f"GBRT R2 for the first 1000 : {r2_oos[1]:.3f}, GBRT R2 for last 1000 : {r3_oos[1]:.3f}")


Special Case: OLS-3

In [ ]:
updated = pd.concat([x_c, x_b], ignore_index=True)
#assume that the 4th column is the Market Value of the stock. Proceeding to arrange it in accordance to the column values
updated = updated.sort_values(by=updated.columns[3], ascending=False)
rtop_1 = updated.iloc[:,:1]
ctop1 = updated.iloc[:,1:4]

rtop_1 = rtop_1.dropna()
ctop1 = ctop1.dropna()

rtop_1.to_csv('r1_1.csv', index=False, header=False)
ctop1.to_csv('c1.csv', index=False, header=False)

In [ ]:
MC = 1
datanum = 50  # Or datanum = 100; separately run two cases
path = './Simu'  # set your own folder path    
dirstock = f"{path}/SimuData_z{datanum}/"

hh = [1]
title = f"{path}/Simu_p{datanum}/Reg{hh[0]}"
if not os.path.exists(title) and MC == 1:
    os.makedirs(title)
titleB = f"{title}/B"
if not os.path.exists(titleB) and MC == 1:
    os.makedirs(titleB)
if datanum == 50:
    nump = 50
elif datanum == 100:
    nump = 100
mu = 0.2 * np.sqrt(hh[0])
tol = 10**(-10)

for M in range(1, 2):  # Assuming range for M
    for mo in range(1, 2):  # Assuming range for mo
        
        print('### MCMC : {}, Model : {} ###'.format(M, mo))
        
        N = 200   # Number of CS tickers
        m = nump * 2   # Number of Characteristics
        T = 180   # Number of Time Periods
        
        per = np.tile(np.arange(1, N+1), T)
        time = np.repeat(np.arange(1, T+1), N)
        stdv = 0.05
        theta_w = 0.005
        
        # Read Files
        path1 = dirstock + 'c' + str(M) + '.csv'
        path2 = dirstock + 'r' + str(mo) + '_' + str(M) + '.csv'
        c = np.genfromtxt(path1, delimiter=',')
        r1 = np.genfromtxt(path2, delimiter=',')
        
        # Add Some Elements
        daylen = np.repeat(N, T//3)
        daylen_test = daylen.copy()
        ind = np.arange(0, N*T//3)
        xtrain = c[ind]
        ytrain = r1[ind]
        trainper = per[ind]
        start_idx = math.floor(N * T / 3) + 1
        end_idx = math.floor(N * (T * 2 / 3 - hh[0] + 1))
        ind = list(range(start_idx, end_idx))
        xtest = c[ind]
        ytest = r1[ind]
        testper = per[ind]
        
        l1 = c.shape[0]
        l2 = len(r1)
        l3 = l2 - np.isnan(r1).sum()
        
        ind = np.arange(N*(2*T//3), min([l1, l2, l3]))
        xoos = c[ind]
        yoos = r1[ind]
        
        # Monthly Demean
        ytrain_demean = ytrain - np.mean(ytrain)
        ytest_demean = ytest - np.mean(ytest)
        mtrain = np.mean(ytrain)
        mtest = np.mean(ytest)
        
        # Calculate Sufficient Stats
        sd = np.zeros(xtrain.shape[1])
        for i in range(xtrain.shape[1]):
            s = np.std(xtrain[:, i])
            if s > 0:
                xtrain[:, i] /= s
                xtest[:, i] /= s
                xoos[:, i] /= s
                sd[i] = s
        
        XX = np.dot(xtrain.T, xtrain)
        U, S, V = np.linalg.svd(XX)
        L = S[0]
        Y = ytrain_demean
        XY = np.dot(xtrain.T, Y)
        
         
        #OLS
        
        # Initialize arrays
        r2_oos = np.zeros(13)
        r2_is = np.zeros(13)
        modeln = 0
        groups = 0
        nc = 0

        # OLS
        modeln += 1
        clf = LinearRegression(fit_intercept=False)
        clf.fit(xtrain, ytrain_demean)
        yhatbig1 = clf.predict(xoos) + mtrain
        r2_oos[modeln - 1] = 1 - np.sum(np.power(yhatbig1 - yoos, 2)) / np.sum(np.power(yoos - mtrain, 2))
        yhatbig1 = clf.predict(xtrain) + mtrain
        r2_is[modeln - 1] = 1 - np.sum(np.power(yhatbig1 - ytrain, 2)) / np.sum(np.power(ytrain - mtrain, 2))
        b = clf.coef_
        pathb = f"{title}/B/b{mo}_{M}_{modeln}.csv"
        pd.DataFrame(b).to_csv(pathb, index=False, header=False)
        print(f"Simple OLS R2: {r2_oos[modeln - 1]:.3f}")

        # OLS+H
        modeln += 1
        func = soft_threshodl
        b = proximalH(groups, nc, xtest, mtrain, ytest, b, xtrain, ytrain_demean, mu, tol, L, 0, func)
        yhatbig1 = np.dot(xoos, b) + mtrain
        r2_oos[modeln - 1] = 1 - np.sum(np.power(yhatbig1 - yoos, 2)) / np.sum(np.power(yoos - mtrain, 2))
        yhatbig1 = np.dot(xtrain, b) + mtrain
        r2_is[modeln - 1] = 1 - np.sum(np.power(yhatbig1 - ytrain, 2)) / np.sum(np.power(ytrain - mtrain, 2))
        pathb = f"{title}/B/b{mo}_{M}_{modeln}.csv"
        pd.DataFrame(b).to_csv(pathb, index=False, header=False)
        print(f"Simple OLS+H R2: {r2_oos[modeln - 1]:.3f}")

In [ ]:
MC = 1
datanum = 50  # Or datanum = 100; separately run two cases
path = './Simu'  # set your own folder path    
dirstock = f"{path}/SimuData_z{datanum}/"

hh = [1]
title = f"{path}/Simu_p{datanum}/Reg{hh[0]}"
if not os.path.exists(title) and MC == 1:
    os.makedirs(title)
titleB = f"{title}/B"
if not os.path.exists(titleB) and MC == 1:
    os.makedirs(titleB)
if datanum == 50:
    nump = 50
elif datanum == 100:
    nump = 100
mu = 0.2 * np.sqrt(hh[0])
tol = 10**(-10)

for M in range(1, 2):  # Assuming range for M
    for mo in range(1, 2):  # Assuming range for mo
        
        print('### MCMC : {}, Model : {} ###'.format(M, mo))
        
        N = 200   # Number of CS tickers
        m = nump * 2   # Number of Characteristics
        T = 180   # Number of Time Periods
        
        per = np.tile(np.arange(1, N+1), T)
        time = np.repeat(np.arange(1, T+1), N)
        stdv = 0.05
        theta_w = 0.005
        
        # Read Files
        path1 = dirstock + 'c' + str(M) + '.csv'
        path2 = dirstock + 'r' + str(mo) + '_' + str(M) + '.csv'
        c = np.genfromtxt(path1, delimiter=',')
        r1 = np.genfromtxt(path2, delimiter=',')
        
        # Add Some Elements
        daylen = np.repeat(N, T//3)
        daylen_test = daylen.copy()
        ind = np.arange(0, N*T//3)
        xtrain = c[ind]
        ytrain = r1[ind]
        trainper = per[ind]
        start_idx = math.floor(N * T / 3) + 1
        end_idx = math.floor(N * (T * 2 / 3 - hh[0] + 1))
        ind = list(range(start_idx, end_idx))
        xtest = c[ind]
        ytest = r1[ind]
        testper = per[ind]
        
        l1 = c.shape[0]
        l2 = len(r1)
        l3 = l2 - np.isnan(r1).sum()
        
        ind = np.arange(N*(2*T//3), min([l1, l2, l3]))
        xoos = c[ind]
        yoos = r1[ind]
        
        # Monthly Demean
        ytrain_demean = ytrain - np.mean(ytrain)
        ytest_demean = ytest - np.mean(ytest)
        mtrain = np.mean(ytrain)
        mtest = np.mean(ytest)
        
        # Calculate Sufficient Stats
        sd = np.zeros(xtrain.shape[1])
        for i in range(xtrain.shape[1]):
            s = np.std(xtrain[:, i])
            if s > 0:
                xtrain[:, i] /= s
                xtest[:, i] /= s
                xoos[:, i] /= s
                sd[i] = s
        
        XX = np.dot(xtrain.T, xtrain)
        U, S, V = np.linalg.svd(XX)
        L = S[0]
        Y = ytrain_demean
        XY = np.dot(xtrain.T, Y)
        
         
        #OLS
        
        # Initialize arrays
        r2_oos = np.zeros(13)
        r2_is = np.zeros(13)
        modeln = 0
        groups = 0
        nc = 0

        # OLS
        modeln += 1
        clf = LinearRegression(fit_intercept=False)
        clf.fit(xtrain, ytrain_demean)
        yhatbig1 = clf.predict(xoos) + mtrain
        r2_oos[modeln - 1] = 1 - np.sum(np.power(yhatbig1[:1000] - yoos[:1000], 2)) / np.sum(np.power(yoos[:1000] - mtrain, 2))
        yhatbig1 = clf.predict(xtrain) + mtrain
        r2_is[modeln - 1] = 1 - np.sum(np.power(yhatbig1[:1000] - ytrain[:1000], 2)) / np.sum(np.power(ytrain[:1000] - mtrain, 2))
        b = clf.coef_
        pathb = f"{title}/B/b{mo}_{M}_{modeln}.csv"
        pd.DataFrame(b).to_csv(pathb, index=False, header=False)
        print(f"Simple OLS R2: {r2_oos[modeln - 1]:.3f}")

        # OLS+H
        modeln += 1
        func = soft_threshodl
        b = proximalH(groups, nc, xtest, mtrain, ytest, b, xtrain, ytrain_demean, mu, tol, L, 0, func)
        yhatbig1 = np.dot(xoos, b) + mtrain
        r2_oos[modeln - 1] = 1 - np.sum(np.power(yhatbig1[:1000] - yoos[:1000], 2)) / np.sum(np.power(yoos[:1000] - mtrain, 2))
        yhatbig1 = np.dot(xtrain, b) + mtrain
        r2_is[modeln - 1] = 1 - np.sum(np.power(yhatbig1[:1000] - ytrain[:1000], 2)) / np.sum(np.power(ytrain[:1000] - mtrain, 2))
        pathb = f"{title}/B/b{mo}_{M}_{modeln}.csv"
        pd.DataFrame(b).to_csv(pathb, index=False, header=False)
        print(f"Simple OLS+H R2: {r2_oos[modeln - 1]:.3f}")

In [ ]:
MC = 1
datanum = 50  # Or datanum = 100; separately run two cases
path = './Simu'  # set your own folder path    
dirstock = f"{path}/SimuData_z{datanum}/"

hh = [1]
title = f"{path}/Simu_p{datanum}/Reg{hh[0]}"
if not os.path.exists(title) and MC == 1:
    os.makedirs(title)
titleB = f"{title}/B"
if not os.path.exists(titleB) and MC == 1:
    os.makedirs(titleB)
if datanum == 50:
    nump = 50
elif datanum == 100:
    nump = 100
mu = 0.2 * np.sqrt(hh[0])
tol = 10**(-10)

for M in range(1, 2):  # Assuming range for M
    for mo in range(1, 2):  # Assuming range for mo
        
        print('### MCMC : {}, Model : {} ###'.format(M, mo))
        
        N = 200   # Number of CS tickers
        m = nump * 2   # Number of Characteristics
        T = 180   # Number of Time Periods
        
        per = np.tile(np.arange(1, N+1), T)
        time = np.repeat(np.arange(1, T+1), N)
        stdv = 0.05
        theta_w = 0.005
        
        # Read Files
        path1 = dirstock + 'c' + str(M) + '.csv'
        path2 = dirstock + 'r' + str(mo) + '_' + str(M) + '.csv'
        c = np.genfromtxt(path1, delimiter=',')
        r1 = np.genfromtxt(path2, delimiter=',')
        
        # Add Some Elements
        daylen = np.repeat(N, T//3)
        daylen_test = daylen.copy()
        ind = np.arange(0, N*T//3)
        xtrain = c[ind]
        ytrain = r1[ind]
        trainper = per[ind]
        start_idx = math.floor(N * T / 3) + 1
        end_idx = math.floor(N * (T * 2 / 3 - hh[0] + 1))
        ind = list(range(start_idx, end_idx))
        xtest = c[ind]
        ytest = r1[ind]
        testper = per[ind]
        
        l1 = c.shape[0]
        l2 = len(r1)
        l3 = l2 - np.isnan(r1).sum()
        
        ind = np.arange(N*(2*T//3), min([l1, l2, l3]))
        xoos = c[ind]
        yoos = r1[ind]
        
        # Monthly Demean
        ytrain_demean = ytrain - np.mean(ytrain)
        ytest_demean = ytest - np.mean(ytest)
        mtrain = np.mean(ytrain)
        mtest = np.mean(ytest)
        
        # Calculate Sufficient Stats
        sd = np.zeros(xtrain.shape[1])
        for i in range(xtrain.shape[1]):
            s = np.std(xtrain[:, i])
            if s > 0:
                xtrain[:, i] /= s
                xtest[:, i] /= s
                xoos[:, i] /= s
                sd[i] = s
        
        XX = np.dot(xtrain.T, xtrain)
        U, S, V = np.linalg.svd(XX)
        L = S[0]
        Y = ytrain_demean
        XY = np.dot(xtrain.T, Y)
        
         
        #OLS
        
        # Initialize arrays
        r2_oos = np.zeros(13)
        r2_is = np.zeros(13)
        modeln = 0
        groups = 0
        nc = 0

        # OLS
        modeln += 1
        clf = LinearRegression(fit_intercept=False)
        clf.fit(xtrain, ytrain_demean)
        yhatbig1 = clf.predict(xoos) + mtrain
        r2_oos[modeln - 1] = 1 - np.sum(np.power(yhatbig1 - yoos, 2)) / np.sum(np.power(yoos - mtrain, 2))
        yhatbig1 = clf.predict(xtrain) + mtrain
        r2_is[modeln - 1] = 1 - np.sum(np.power(yhatbig1 - ytrain, 2)) / np.sum(np.power(ytrain - mtrain, 2))
        b = clf.coef_
        pathb = f"{title}/B/b{mo}_{M}_{modeln}.csv"
        pd.DataFrame(b).to_csv(pathb, index=False, header=False)
        print(f"Simple OLS R2: {r2_oos[modeln - 1]:.3f}")

        # OLS+H
        modeln += 1
        func = soft_threshodl
        b = proximalH(groups, nc, xtest, mtrain, ytest, b, xtrain, ytrain_demean, mu, tol, L, 0, func)
        yhatbig1 = np.dot(xoos, b) + mtrain
        r2_oos[modeln - 1] = 1 - np.sum(np.power(yhatbig1[-1000:] - yoos[-1000:], 2)) / np.sum(np.power(yoos[-1000:] - mtrain, 2))
        yhatbig1 = np.dot(xtrain, b) + mtrain
        r2_is[modeln - 1] = 1 - np.sum(np.power(yhatbig1[-1000:] - ytrain[-1000:], 2)) / np.sum(np.power(ytrain[-1000:] - mtrain, 2))
        pathb = f"{title}/B/b{mo}_{M}_{modeln}.csv"
        pd.DataFrame(b).to_csv(pathb, index=False, header=False)
        print(f"Simple OLS+H R2: {r2_oos[modeln - 1]:.3f}")